# Football Parsing Code

In [2]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))

from os.path import join
from fsUtils import mkSubDir, setFile, isFile, removeFile, isDir
from ioUtils import getFile, saveFile
from timeUtils import clock, elapsed
from fileUtils import getBaseFilename, getBasename, getDirname
from webUtils import getWebData, getHTML
from timeUtils import printDateTime, getDateTime, addMonths
from searchUtils import findExt
from time import sleep
from random import random
import sys
import re
from datetime import timedelta

from espngames import output, espn
from espngames import game, team, season
from espngames import historical

import datetime as dt
start = dt.datetime.now()

print("Notebook Last Run Initiated: "+str(start))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Python: 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:07:29) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2019-04-04 16:54:35.134861


# Order To Download ESPN Data

In [3]:
hist = historical()
#hist.downloadTeamStandings()
#hist.parseAndDownloadTeamYearlyStandings()
#hist.parseTeamYearlyStandings()
#hist.downloadGameData()
#hist.parseGameData()

# Extract Game Data

In [ ]:
class penalty:
    def __init__(self, text=None):
        self.text = text
        
        yards = None
        penaltytype = None
        
        ## False Start
        if sum([x in self.text for x in ["False Start"]]) > 0:
            yards = -5
            penaltytype = "FalseStart"
            
        ## Offensive Pass Interference
        if sum([x in self.text for x in ["Offensive Pass Interference"]]) > 0:
            yards = "NET"
            penaltytype = "OffensivePassInterference"
            
        ## Defensive Pass Interference
        if sum([x in self.text for x in ["Defensive Pass Interference"]]) > 0:
            yards = "NET"
            penaltytype = "DefensivePassInterference"
            
        ## Defensive Pass Interference
        if sum([x in self.text for x in ["Defensive Offside"]]) > 0:
            yards = 5
            penaltytype = "DefensiveOffside"
            
        ## Offensive Pass Interference
        if sum([x in self.text for x in ["Illegal Block"]]) > 0:
            yards = "NET"
            penaltytype = "IllegalBlock"

        self.yards = yards
        self.penaltytype = penaltytype
        
    def isPenalty(self, returnYards=False):
        if self.penaltytype is None:
            if returnYards:
                return 0
            else:
                return False
        else:
            if returnYards:
                return self.yards
            else:
                return True

In [10]:
############################################################################################################
## Play Text Yardage Play
############################################################################################################
class playyards:
    def __init__(self, text=None, playtype=None, test=False):
        self.text     = text
        self.playtype = playtype
        self.test     = test
        
        self.playyards    = None
        self.kickingyards = None
        self.runbackyards = None
        self.penaltyyards = None
        
        self.yards = None
        
    def setPlay(self, text):
        self.text = text
        
    def setPlayType(self, playtype):
        self.playtype = playtype
        
    def makeYards(self, yards, debug=False):
        if self.test:
            return yards
        
        if yards == "NET" or yards == "IGN":
            return yards
        
        if yards == None:
            return None
        
        try:
            result = int(yards)
        except:
            raise ValueError("Could not extract yards from [{0}] play with [{1}] text".format(yards, self.text))
            
        return result
        
        
    def show(self, yards, caller, debug=False):            
        if debug:
            print("\t\t===> {0} Yards -> [{1}]".format(caller, yards))
            
    
    ############################################################################################################
    ## Find Play Yardage
    ############################################################################################################
    def findYards(self, debug=False):
        sign   = 1
        result = None
        
        self.findPenaltyYards(debug)
        self.findKickingYards(debug)
        self.findReturnYards(debug)
        self.findPlayYards(debug)
        
        yards = [self.playyards, self.kickingyards, self.runbackyards, self.penaltyyards]
        if not len([x for x in yards if x is not None]):
            if self.test is True:
                if debug:
                    print("Could not find yards in this play")
                self.yards = None
            else:
                raise ValueError("Could not find any yards for this play: [{0}]".format(self.text))
        else:
            if any([isinstance(x, int) for x in yards]):
                self.yards = sum([x for x in yards if (x is not None) and (isinstance(x, int))])
            elif any([isinstance(x, str) for x in yards]):
                self.yards = ", ".join([x for x in yards if (x is not None) and (isinstance(x, str))])
            else:
                self.yards = None
            


    
    ############################################################################################################
    ## Find Play Yardage
    ############################################################################################################
    def findPlayYards(self, debug=False):
        sign   = 1
        result = None
        start  = self.text
        
        prep = ("(or|for)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
        
        #################################################################
        ### Check for yards
        #################################################################
        if result is None:
            if sum([x in self.text for x in ["no gain", "no loss", "DECLINED"]]) > 0:
                result = [0]
                if debug:
                    print("\t\t===> {0}".format(result))      

        if result is None:
            if sum([x in self.text for x in ["loss of zero", "gain of zero"]]) > 0:
                result = [0]
                if debug:
                    print("\t\t===> {0}".format(result))                

        if result is None:
            m = re.search(r"{0}\s{1}\s{2}".format(prep, num, dist), self.text)
            if m is not None:
                result = m.groups(0)
                result = result[1:]
                if debug:
                    print("\t\t===> {0}".format(result))        
        
        if result is None:
            m = re.search(r"\w+\s(-\d+)\s\w+", self.text)
            if m is not None:
                result = m.groups(0)                
                if debug:
                    print("\t\t===> {0}".format(result))

        if result is None:
            m = re.search(r"for\sa\s(\w+)\sof\s(\d+)\s{0}".format(dist), self.text)
            if m is not None:
                results = m.groups()
                sign    = results[0]
                result  = results[1:]
                if sign == "loss":
                    sign = -1
                if debug:
                    print("\t\t===> {0} ({1})".format(result, sign))
        
        if result is None:
            pattern = r"{0}\s{1}\spenalty".format(num, dist)
            m = re.search(pattern, self.text)
            if m is not None:
                sign   = -1
                result = m.groups()
                if debug:
                    print("\t\t===> {0}".format(result))
        
        if result is None:
            if sum([x in self.text for x in ["incomplete", "Incomplete"]]) > 0:
                result = [0]
                if debug:
                    print("\t\t===> {0}".format(result))
        
        if result is None:
            if sum([x in self.text for x in [" no gain", " No Gain", "failed"]]) > 0:
                result = [0]
                if debug:
                    print("\t\t===> {0}".format(result))
        
        if result is None:
            m = re.search(r"{0}\s{1}".format(num, dist), self.text)
            if m is not None:
                result = m.groups()
                if debug:
                    print("\t\t===> {0}".format(result))

        if result is None:
            result = [None]
        self.playyards = self.makeYards(result[0])
        self.show(self.runbackyards, "Drive", debug=debug)

        
        

    ############################################################################################################
    ## Find Penalty Yardage
    ############################################################################################################
    def findPenaltyYards(self, debug=False):
        kick = ("(punt|kickoff)")
        prep = ("(or|for)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
        yards = None
        
        ## False Start
        if sum([x in self.text for x in ["False Start"]]) > 0:
            yards = -5
            
        ## Offensive Pass Interference
        if sum([x in self.text for x in ["Offensive Pass Interference"]]) > 0:
            yards = "NET"
            
        ## Offensive Pass Interference
        if sum([x in self.text for x in ["Defensive Pass Interference"]]) > 0:
            yards = "NET"
            
        ## Offensive Pass Interference
        if sum([x in self.text for x in ["Illegal Block"]]) > 0:
            yards = "NET"


        if yards is None:
            yards = "IGN"
        
            

        self.penaltyyards = self.makeYards(yards)
        self.show(self.runbackyards, "Penalty", debug=debug)


        
    ############################################################################################################
    ## Find Kicking Yardage
    ############################################################################################################
    def findKickingYards(self, debug=False):
        kick = ("(punt|kickoff)")
        prep = ("(or|for)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
        yards = None
        
        if yards is None:
            m = re.search(r"{0}\s{1}\s{2}\s{3}".format(kick, prep, num, dist), self.text)
            if m is not None:
                result = m.groups(0)
                yards  = result[2]

        self.kickingyards = self.makeYards(yards)
        self.show(self.kickingyards, "Kicking", debug=debug)


        
    

    ############################################################################################################
    ## Find Return Yardage
    ############################################################################################################
    def findReturnYards(self, debug=False):
        runback  = "(return|runback|returns)"
        norun    = "(downed|down)"
        prep     = "(or|for|at)"
        article  = "(the|a)"
        num      = "([+-?]\d+|\d+)"
        word     = "(\w+)"
        dist     = "(yards|yard|Yds|yds|Yd|yd)"
        
        yards = None
            
        if yards is None:
            m = re.search(r"{0}\s{1}\s{2}\s{3}".format(runback, prep, num, dist), self.text)
            if m is not None:
                result = m.groups(0)
                yards  = result[2]
        
        if yards is None:
            m = re.search(r"{0}\s{1}\s{2}\s{3}\s{4}".format(norun, prep, article, word, num), self.text)
            if m is not None:
                result = m.groups(0)
                yards  = 0
                    
        if yards is None:
            if sum([x in self.text for x in ["fair catch", "Fair Catch"]]) > 0:
                yards = 0

        self.runbackyards = self.makeYards(yards)
        self.show(self.runbackyards, "Return", debug=debug)
                        
            
_ = clock("Last Run")

Current Time is Thu Apr 04, 2019 16:54:59 for Last Run


In [11]:
yds = playyards(test=False)
yds.setPlay("Alex Starzyk punt for 43 yds, downed at the Vandy 21")
yds.setPlay("VANDERBILT Penalty, Offensive Pass Interference (Ralph Webb) to the Vandy 15")
yds.setPlay("P.J. Walker pass incomplete to Brandon Shippen")
yds.setPlay("kickoff for 58 yds")
print(yds.text)
yds.findYards(debug=True)
print("Yards: {0}".format(yds.yards))

kickoff for 58 yds
		===> Penalty Yards -> [None]
		===> Kicking Yards -> [58]
		===> Return Yards -> [None]
		===> ('58', 'yds')
		===> Drive Yards -> [None]
Yards: 116


In [12]:
############################################################################################################
### ## Football Play Data
############################################################################################################
class footballplay:
    def __init__(self, playtype, text, debug=False):
        self.playtype = playtype.title()
        self.text     = text
        
        self.__dict__[playtype] = True
        
        self.startY  = None
        self.endY    = None
        self.startT  = None
        self.endT    = None
        
        self.down    = None
        self.togo    = None
        self.quarter = None
        
        self.result   = None
        self.gain     = None
        self.turnover = None
        self.newdowns = None
        
        self.possession = None

        self.printType(debug)
        
    def setType(self, playtype):
        self.type = playtype
        
    def printType(self, debug=False):
        if debug:
            print("===> {0} Play".format(self.playtype))
        
        
     
        
        
class playanalysis:
    def __init__(self, text, playtype):
        self.text  = text
        self.yards = None
        
        self.__dict__[playtype] = True
        self.playtype = playtype
        
        
        py = playyards(text=text, playtype=playtype)
        self.findYards = py.findYards
        
        self.called = {}
    
    
    def setPlayText(self, text):
        self.text = text
    
    
    def getKey(self, key):
        if self.__dict__.get(key) is not None:
            return self.__dict__[key]
        return None
    
    
    def setNoPlay(self):
        if self.getKey("noplay") is True:
            for k in self.__dict__.keys():
                if self.__dict__[k] is True:
                    self.__dict__[k] = False
            self.noplay = True
        
    
    def isScore(self, points=True):
        if self.getKey("noplay") is True:
            if points:
                return 0
            return False
        
        if self.getKey("touchdown") is True:
            if points:
                return 6
            return True
        elif self.getKey("fieldgoal") is True:
            if points:
                return 3
            return True
        elif self.getKey("safetypts") is True:
            if points:
                return 2
            return True
        elif self.getKey("pat") is True:
            if self.getKey("oneptpat") is True:
                if points:
                    return 1
                return True
            elif self.getKey("twoptpat") is True:
                if points:
                    return 2
                return True
            elif self.getKey("defpat") is True:
                if points:
                    return 2
                return True
            else:
                if points:
                    return 0
                return False
        else:
            if points:
                return 0
            return False
            
        
    ##################################################
    ## Passing Play
    ##################################################
    def findInterceptedPass(self, debug): 
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.interception = False
        if sum([x in self.text for x in ["interception", "Interception", "intercepted", "Intercepted"]]) > 0:
            self.interception = True

            if self.touchdown is False:
                if sum([x in self.text for x in [" Kick)", " KICK)"]]) > 0:
                    self.touchdown = True

        
    def findInterceptedPAT(self, debug):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        try:
            patsubplay = self.play[self.text.rfind("("):self.text.rfind(")")]
            self.interception = False
            if sum([x in patsubplay for x in ["interception", "Interception", "intercepted", "Intercepted"]]) > 0:
                self.interception = True
        except:
            if self.getKey("interception") is None:
                self.interception = False
            return
        
        

    def findPassing(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay(debug)
        
        self.complete = False
        self.interception = False
        
        if sum([x in self.text for x in ["complete", "Complete"]]) > 0:
            self.complete = True
                        
        if self.complete is False:
            self.findInterceptedPass(debug)
                    
        
    ##################################################
    ## Rushing Play
    ##################################################
    def findRushing(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay(debug)
                    
        
    ##################################################
    ## Interception Play
    ##################################################
    def findInterception(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay(debug)
        self.findPassing(debug)
        
        ## Check for implied touchdown
        if self.interceptionreturn is True:
            if self.touchdown is False:
                if sum([x in self.text for x in [" Kick)", " KICK)"]]) > 0:
                    self.touchdown = True
                    
        
    ##################################################
    ## Fumble Play
    ##################################################
    def findFumble(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay(debug)
        
        ## Check for implied touchdown
        if self.fumble is True:
            if self.touchdown is False:
                if sum([x in self.text for x in [" Kick)", " KICK)"]]) > 0:
                    self.touchdown = True
            
    
            
    ##################################################
    ## Basic Play
    ##################################################
    def addPAT(self, debug=False):
        self.addpat = False
        if self.touchdown is True:
            if sum([x in self.text for x in [" Kick)", " KICK)", " kick)", "2 point", " 2 Point", "Two-point", "two-point", "Two-Point"]]) > 0:
                self.addpat = True
                                

    ##################################################
    ## Basic Play
    ##################################################
    def findBasicPlay(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True


        self.fumble    = False
        self.touchdown = False
        self.safety    = False
        self.penalty   = False
        self.runback   = False
        self.noplay    = False


        if sum([x in self.text for x in ["touchdown", "Touchdown", "TOUCHDOWN", " TD", "TD,", "TD "]]) > 0:
            ## Noted Exceptions
            if self.text.find("TD Moton") != -1:
                self.touchdown = False
            else:
                self.touchdown = True
            
        if sum([x in self.text for x in [" fumble", " Fumble", " fumbled", " Fumbled"]]) > 0:
            self.fumble = True
            
        if sum([x in self.text for x in ["penalty", "PENALTY", "Penalty"]]) > 0:
            self.penalty = True
        
        if sum([x in self.text for x in ["return", "Return"]]) > 0:
            self.runback = True
            
        if sum([x in self.text for x in ["recovered", "Recovered"]]) > 0:
            self.runback = True
            
        if sum([x in self.text for x in ["NO PLAY", "No Play"]]) > 0:
            self.noplay = True
        
        if sum([x in self.text for x in ["SAFETY", "Safety"]]) > 0:
            self.safety = True
        
            
    ##################################################
    ## Basic Kicking
    ##################################################
    def findBasicKicking(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay()
        self.blocked   = False
        self.missed    = False
        self.runback   = False
        self.touchback = False
        
        
        if sum([x in self.text for x in ["return", "Return"]]) > 0:
            self.runback = True
        
        if sum([x in self.text for x in ["no return", "No Return"]]) > 0:
            self.runback = False

        if sum([x in self.text for x in [" block", " blocked", "Blocked", "Block", "BLOCKED"]]) > 0:
            if sum([x in self.text.upper() for x in ["ILLEGAL BLOCK"]]) == 0:
                self.blocked = True

        if sum([x in self.text for x in ["touchback", "Touchback"]]) > 0:
            self.touchback = True
            
        if sum([x in self.text for x in [" miss ", " missed", " Missed", "MISSED", "no good", "no good", "No Good", "No good", "NO GOOD", "Failed", "FAILED", "failed"]]) > 0:
            self.missed = True
                    
        
    ##################################################
    ## Punt Play
    ##################################################
    def findPunt(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay(debug)
        
        ## Check for implied touchdown
        if self.punt is True:
            if self.touchdown is False:
                if sum([x in self.text for x in [" Kick)", " KICK)"]]) > 0:
                    self.touchdown = True
            
        
    def findFieldGoal(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicKicking(debug)

        if self.blocked is True or self.missed is True:
            self.fieldgoaltry = True
            self.fieldgoal    = False
        else:
            self.fieldgoaltry = False
            self.fieldgoal    = True
        
        
    def findPAT(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicKicking(debug)
        self.findInterceptedPAT(debug)
        
        
        ## Check for incorrect blocked assignment
        if self.blocked is True:
            if sum([x in self.text for x in ["punt", " PUNT", "Punt"]]) > 0:
                self.blocked = False
            if sum([x in self.text for x in ["Field Goal", "FIELD GOAL", "FG"]]) > 0:
                self.blocked = False
                
            
        ## Extra Point
        if self.missed is True or self.blocked is True:
            self.oneptpattry = True
            self.oneptpat    = False
        else:
            self.oneptpattry = False
            self.oneptpat    = True

        
        ## Defensive PAT
        self.defpat = False
        if sum([x in self.text for x in ["defensive PAT", "Defensive PAT"]]) > 0:
            self.defpat      = True
            self.oneptpat    = False
            self.oneptpattry = True

        
        ## 2-Point
        self.twoptpattry = False
        self.twoptpat    = False
        if sum([x in self.text for x in ["2 point", " 2 Point", "Two-point", "two-point", "Two-Point"]]) > 0:
            self.twoptpattry = True
            self.oneptpattry = False
            self.oneptpat    = False
            
            if self.missed is True or self.interception is True:
                self.twoptpat = False
            else:
                self.twoptpat = True
                
        
        ## Interception (from two point)
        if self.interception is True:
            self.twoptpattry = True
            self.twoptpat    = False
            self.oneptpattry = False
            self.oneptpat    = False
            

        ## Sanity check
        self.pat = False
        if sum([self.oneptpat, self.oneptpattry, self.twoptpat, self.twoptpattry, self.defpat]) > 0:
            self.pat = True

            
        
    def findKickoff(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicKicking(debug)
        
        self.touchback = False
        self.onside    = False
        self.recovery  = False
        
        if sum([x in self.text for x in ["onside ", "Onside ", "on-side"]]) > 0:
            self.onside = True

        if sum([x in self.text for x in ["recovered"]]) > 0:
            self.recovery = True

        

    ############################################################################################################
    ## Return Play
    ############################################################################################################
    def findReturn(self, debug=False):
        fname = sys._getframe().f_code.co_name
        if self.called.get(fname) is not None:
            return
        self.called[fname] = True

        self.findBasicPlay()
        
        self.puntreturn         = False
        self.fieldgoalreturn    = False
        self.kickoffreturn      = False
        self.onsidereturn       = False
        self.fumblereturn       = False
        self.interceptionreturn = False

        if self.__dict__.get("punt") is not None:
            if self.punt is True:
                self.puntreturn = True
                self.findPunt()
                
        if self.__dict__.get("kickoff") is not None:
            if self.kickoff is True:
                self.kickoffreturn = True
                
        if self.__dict__.get("onside") is not None:
            if self.onside is True:
                self.onsidereturn = True
                
        if self.__dict__.get("fieldgoal") is not None:
            if self.fieldgoal is True:
                self.fieldgoalreturn = True
                
        if self.__dict__.get("fumble") is not None:
            if self.fumble is True:
                self.fumblereturn = True
                self.findFumble()
                
        if self.__dict__.get("interception") is not None:
            if self.interception is True:
                self.interceptionreturn = True
                self.findInterception()
                

                
        
            
            

    ############################################################################################################
    ## Result of Play
    ############################################################################################################
    def findResult(self, debug=False):
        return self.playtype
        
        self.result = None
        if self.__dict__.get("penalty"):
            if self.penalty is True:
                self.result = "Penalty"
        elif self.__dict__.get("timeout"):
            if self.timeout is True:
                self.result = "Timeout"
        elif self.__dict__.get("end"):
            if self.end is True:
                self.result = "End"
        elif self.__dict__.get("touchdown"):
            if self.touchdown is True:
                self.result = "Touchdown"
        elif self.__dict__.get("madefieldgoal"):
            if self.madefieldgoal is True:
                self.result = "FieldGoal"
        elif self.__dict__.get("madefieldpat"):
            if self.madefieldpat is True:
                self.result = "PAT"
        elif self.__dict__.get("interception"):
            if self.interception is True:
                self.result = "Interception"
        elif self.__dict__.get("punt"):
            if self.punt is True:
                self.result = "Punt"
        else:
            self.result = "Continue"


_, _ = clock("Last Run")

Current Time is Thu Apr 04, 2019 16:55:03 for Last Run


In [257]:
############################################################################################################
## Penalty Play
############################################################################################################
class penaltyplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "penalty"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa  = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Penalty Specific
        self.penaltytype  = None
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = self.yds.penaltyyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is penaltyyards=={1}".format(self.name, self.result))
            
            
        
############################################################################################################
## Punt Play
############################################################################################################
class puntplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "punt"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa  = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.kicker = None
                
    def analyze(self, debug=False):
        self.pa.findPunt()
        self.yds.findYards()
        self.result = self.yds.kickingyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for kickyards=={2} and runbackyards=={3}".format(self.name, self.result, self.yds.kickyards, self.yds.runbackyards))
            print("\tResult of {0} play is touchback=={1}, blocked=={2}, touchdown=={3}".format(self.name, self.pa.touchback, self.pa.blocked, self.pa.touchdown))                    

        
        
############################################################################################################
## Kickoff Play
############################################################################################################
class kickoffplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "kickoff"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa  = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.kicker = None
                
    def analyze(self, debug=False):
        self.pa.findKickoff()
        self.yds.findYards()
        self.result = self.yds.kickingyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for kickyards=={2} and runbackyards=={3}".format(self.name, self.result, self.yds.kickyards, self.yds.runbackyards))
            print("\tResult of {0} play is touchback=={1}, blocked=={2}, touchdown=={3}".format(self.name, self.pa.touchback, self.pa.blocked, self.pa.touchdown))            
        

        
############################################################################################################
## Field Goal Play
############################################################################################################
class fieldgoalplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "fieldgoal"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.kicker = None
                
    def analyze(self, debug=False):
        self.pa.findFieldGoal()
        self.yds.findYards()
        self.result = self.yds.kickingyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
            print("\tResult of {0} play is blocked=={1}, return=={2}".format(self.name, self.blocked, self.runback))
        

        
############################################################################################################
## PAT Play
############################################################################################################
class patplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "pat"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.kicker = None
                
    def analyze(self, debug=False):
        self.pa.findPAT()
        self.yds.findYards()
        self.result = self.yds.kickingyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
            print("\tResult of {0} play is blocked=={1}, return=={2}".format(self.name, self.blocked, self.runback))

            

############################################################################################################
## Return Play
############################################################################################################
class returnplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "return"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa  = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.returner = None
                
    def analyze(self, debug=False):
        self.pa.findBasicKicking()
        self.pa.findBasicPlay()
        self.pa.findPassing()
        self.pa.findRushing()
        self.pa.findReturn()
        self.yds.findYards()
        self.result = self.yds.runbackyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for runbackyards=={2}".format(self.name, self.result, self.yds.runbackyards))
            print("\tResult of {0} play is touchback=={1}, blocked=={2}, touchdown=={3}".format(self.name, self.pa.touchback, self.pa.blocked, self.pa.touchdown))   
        

        
############################################################################################################
## Downs Play
############################################################################################################
class downsplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "downs"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = 0
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## TBD Play
############################################################################################################
class tbdplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "tbd"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.pa.findBasicKicking()
        self.yds.findYards()
        self.result = None
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## No Play
############################################################################################################
class noplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "no"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = 0
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## Safety Play
############################################################################################################
class safetyplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "safety"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)

        ## Safety Specific
        self.rushing = None
        self.passing = None
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = self.yds.yards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## Timeout Play
############################################################################################################
class timeoutplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "timeout"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = 0
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## End Play
############################################################################################################
class endplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "end"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = 0
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## Begin Play
############################################################################################################
class beginplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "begin"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = 0
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## Sack Play
############################################################################################################
class sackplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "sack"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.quarterback = None
                
    def analyze(self, debug=False):
        self.pa.findBasicPlay()
        self.yds.findYards()
        self.result = self.yds.playyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))
        
        


############################################################################################################
## Run Play
############################################################################################################
class rushingplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "rushing"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Running Back
        self.runningback = None
                
    def analyze(self, debug=False):
        self.pa.findRushing()
        self.yds.findYards()
        self.result = self.yds.playyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for yards=={2}".format(self.name, self.result))          

            

        
############################################################################################################
## Pass Play
############################################################################################################
class passingplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "passing"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
        
        ## Players
        self.quarterback = None
        self.receiver    = None
                
    def analyze(self, debug=False):
        self.pa.findPassing()
        self.yds.findYards()
        self.result = self.yds.playyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for passingyards=={2}".format(self.name, self.result))
        

        
############################################################################################################
## Fumble Play
############################################################################################################
class fumbleplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "fumble"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)

        ## Fumble Specific
        self.rushing = None
        self.passing = None
        
        ## Player Specific
        self.fumbler   = None
        self.recoverer = None
        self.turnover  = None
                
    def analyze(self, debug=False):
        self.pa.findReturn()
        self.yds.findYards()
        self.result = self.yds.playyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1} for returnyards=={2}".format(self.name, self.result, self.result))
        
        


############################################################################################################
## Touchdown Play
############################################################################################################
class touchdownplay(footballplay):
    def __init__(self, text, debug=False):
        self.name = "touchdown"
        self.text = text
        footballplay.__init__(self, self.name, text, debug)
        self.pa = playanalysis(text=text, playtype=self.name)
        self.yds = playyards(text=text, playtype=self)
                
    def analyze(self, debug=False):
        self.pa.findReturn()
        self.yds.findYards()
        self.result = self.yds.playyards
        self.show(debug)

    def show(self, debug=False):
        if debug:
            print("\tResult of {0} play is result=={1}".format(self.name, self.result))
        
            
_ = clock("Last Run")

Current Time is Thu Apr 04, 2019 22:05:00 for Last Run


In [258]:
class playstartclass:
    def __init__(self, down=None, togo=None, startY=None, side=None, extra=None):
        self.down   = down
        self.togo   = togo
        self.startY = startY
        self.side   = side
        self.extra  = extra
        self.quarter   = None
        self.gameclock = None
        self.distToEndZone = None
        
    def setStartY(self, startY):
        self.startY = startY
        
    def setDistToEndZone(self, dist):
        self.distToEndZone = dist
        
    def setQuarter(self, quarter):
        self.quarter = quarter
        
    def setGameClock(self, gameclock):
        self.gameclock = gameclock
        
        
class drivedetailclass:
    def __init__(self, plays, yards, gametime):
        self.plays = plays
        self.yards = yards
        self.gametime = gametime
        
        
class playpossessionclass:
    def __init__(self, start, end, text=None):
        self.start = start
        self.end   = end
        
        self.text  = text
        
        self.valid = True
        
        self.prev    = "PREV"
        self.unknown = "UNK"
        
    def isPreviousStart(self):
        return self.start == self.prev
        
    def setPreviousStart(self):
        self.start  = self.prev
        
    def isUnknownStart(self):
        return self.start == self.unknown
        
    def setUnknownStart(self):
        self.start  = self.unknown
        
        
class playclass:
    def __init__(self, possession, start, play, valid):
        self.possession = possession
        self.start      = start
        self.play       = play
        self.valid      = valid
        

class driveclass:
    def __init__(self, headline, detail, possession, postdrivehomescore, postdriveawayscore, plays=None):
        self.headline = headline
        self.detail   = detail
        self.possession = possession
        self.postdrivehomescore = postdrivehomescore
        self.postdriveawayscore = postdriveawayscore
        self.plays = plays
        
    def setPlays(self, plays):
        self.plays = plays
        
        
class playtype:
    def __init__(self, playText, debug=False):
        play = None
        self.playtext = playText
        self.valid = True
        self.unknownPlays = []
        
        playSubText = playText
        pos = playText.rfind("(")
        if pos != -1 and pos > 5:
            playSubText = playText[:pos].strip()
            
        
        if sum([x in playSubText for x in ["field goal", "Field Goal", "Field goal", "FIELD GOAL", " fg ", " FG ", "MISSED FG", "Missed FG"]]) > 0:
            play = fieldgoalplay(playText, debug=debug)
        elif sum([x in playText for x in ["fake field goal", "Fake Field Goal"]]) > 0:
            play = fieldgoalplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["timeout", "Timeout"]]) > 0:
            play = timeoutplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["kickoff", "Kickoff", "kick off", "Kick Off", "Onside Kick", "Onside kick", "onside kick", "on-side kick"]]) > 0:
            play = kickoffplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["punt ", "PUNT", "Punt", " punted", " Punted", " punt,", " punter ", "punts "]]) > 0:
            play = puntplay(playText, debug=debug)
        elif playSubText.endswith(" punt"):
            play = puntplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["end of the ", "End Of The ", "End of the ", "end of ", "End of ", "END OF "]]) > 0:
            play = endplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["start of the ", "Start Of The ", "Start of the ", "Begin Drive", "start of ", "Start of ", "START OF "]]) > 0:
            play = beginplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["SAFETY", "safety", "Safety"]]) > 0:
            play = safetyplay(playText, debug=debug)
        elif sum([x in playSubText for x in [" sack ", " sacked ", " Sack ", " Sacked ", "sacked,"]]) > 0:
            play = sackplay(playText, debug=debug)
        elif sum([x in playSubText for x in [" pass ", " passed ", " Pass ", " Passed ", " spikes ", " spiked ", " pass,to ",
                                          "Interception Return", "Interception", "INTERCEPTION"]]) > 0:
            if sum([x in playSubText for x in ["touchdown", "Touchdown", "TOUCHDOWN"]]) > 0:
                play = touchdownplay(playText, debug=debug)
            elif sum([x in playSubText for x in ["penalty", "PENALTY", "Penalty", "Penalty,"]]) > 0:
                play = penaltyplay(playText, debug=debug)
            else:
                play = passingplay(playText, debug=debug)
        elif sum([x in playSubText for x in [" rush ", " rushed ", " Rush ", " Rushed ", " scramble", " rush,", " run ", " Run"]]) > 0:
            if sum([x in playSubText for x in ["touchdown", "Touchdown", "TOUCHDOWN"]]) > 0:
                play = touchdownplay(playText, debug=debug)
            elif sum([x in playSubText for x in ["penalty", "PENALTY", "Penalty", "Penalty,"]]) > 0:
                play = penaltyplay(playText, debug=debug)
            else:
                play = rushingplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["penalty", "PENALTY", "Penalty", "Penalty,"]]) > 0:
            play = penaltyplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["fumble", "Fumble", " fumbled ", " Fumble ", " Fumbled ", "FUMBLE"]]) > 0:
            if sum([x in playSubText for x in ["touchdown", "Touchdown", "TOUCHDOWN"]]) > 0:
                play = touchdownplay(playText, debug=debug)
            else:
                play = fumbleplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["downs", "Downs", "DOWNS"]]) > 0:
            play = downsplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["touchdown", "Touchdown", "TOUCHDOWN"]]) > 0:
            play = touchdownplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["extra point", "Extra Point", "Extra point", "PAT", "2 point", "2 Point", "two-point", "Two-point", "Two-Point"]]) > 0:
            play = patplay(playText, debug=debug)
        elif sum([x in playSubText for x in ["NO PLAY", "No Play"]]) > 0:
            play = noplay(playText, debug=debug)
        elif len(playText) > 1:
            play = tbdplay(playText, debug=debug)
        elif len(playText) <= 1:
            self.play  = None
            self.valid = False
        else:
            self.unknownPlays.append(playText)
            self.play = None
            self.valid = False
            
        self.play = play
        
                
    def isValid(self):
        return self.valid
            
    def getPlay(self):
        return self.play
        
    def getPlayText(self):
        return self.playtext
                        
            
_ = clock("Last Run")

Current Time is Thu Apr 04, 2019 22:05:02 for Last Run


In [259]:
playText = "(OT) Terry Wilson pass complete to Lynn Bowden Jr. for 4 yds to the TexAM 21"
ptype = playtype(playText)
print(ptype.isValid())
play  = ptype.getPlay()
print(play)
playText = "No Play"
ptype = playtype(playText)
print(ptype.isValid())
play  = ptype.getPlay()
print(play)
playText = "Nathan punt"
ptype = playtype(playText)
print(ptype.isValid())
play  = ptype.getPlay()
print(play)
playText = "?"
ptype = playtype(playText)
print(ptype.isValid())
play  = ptype.getPlay()
print(play)
playText = "???"
ptype = playtype(playText)
print(ptype.isValid())
play  = ptype.getPlay()
print(play)

True
True
True
False
None
True


In [268]:
class playbyplay(espn, output):
    def __init__(self):
        self.name = "playbyplay"
        espn.__init__(self)
        output.__init__(self)
        
        self.hist = None
        
        
    def setHistorical(self, hist):
        self.hist = hist
    
    
    def parsePlayStart(self, playStartText, playNo, numPlays, playData):
        if len(playStartText) == 0:
            return None
        
        ps = None
        
        down = "(st|nd|rd|th)"
        num  = "(\d+)"
        word = "(\w+)"
        goal = "(GOAL|goal|Goal)"
        prep = "(on|at)"
        amp  = "(&|and)"
        
        ## Look for match of type [1st and 10 at CAL 19]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, num, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3]
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for match of type [1st and GOAL at CAL 1]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}\s{6}".format(num, down, amp, goal, prep, word, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[6] ## replace goal with distance to goal
            startY = groups[6]
            side   = groups[5]
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        ## Look for very near match without side of field [1st and 10 at 50]
        m = re.search(r"{0}{1}\s{2}\s{3}\s{4}\s{5}".format(num, down, amp, goal, prep, num), playStartText)
        if m is not None:
            groups = m.groups()
            down   = groups[0]
            togo   = groups[3] ## replace goal with distance to goal
            startY = groups[5]
            side   = None
            ps     = playstartclass(down=down, togo=togo, startY=startY, side=side)
            

        return ps
        raise ValueError("Could not parse [{0}] and [{1}]".format(playStartText, playData))
        
        
        
    
    def parseClockAndQuarter(self, playData, debug=False):
        gameclock = None
        quarterNo = None
        
        ## Check for end of quarter/half/game
        if sum([x in playData for x in ["End of the ", "End Of The ", "end of the ", "end of ", "End of "]]) > 0:
            gameclock = timedelta(minutes=0, seconds=0) 
            if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                quarterNo = 1
            elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half", "half", "Half"]]) > 0:
                quarterNo = 2
            elif sum([x in playData for x in ["3rd Quarter", "3rd quarter"]]) > 0:
                quarterNo = 3
            elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                quarterNo = 4
            elif "OT" in playData:
                quarterNo = "OT"
            else:
                quarterNo = "UNKNOWN END QTR"
                #raise ValueError("Could not determine quarter from {0}".format(playData))

                
        if gameclock is None and quarterNo is None:
            ## Check for start of quarter/half/game
            if sum([x in playData for x in ["Start of the ", "Start Of The ", "start of the ", "start of "]]) > 0:
                gameclock = timedelta(minutes=15, seconds=0) 
                if sum([x in playData for x in ["1st Quarter", "1st quarter"]]) > 0:
                    quarterNo = 1
                elif sum([x in playData for x in ["2nd Quarter", "2nd quarter", "1st Half", "1st half"]]) > 0:
                    quarterNo = 2
                elif sum([x in playData for x in ["3rd Quarter", "3rd quarter", "2nd Half", "2nd half"]]) > 0:
                    quarterNo = 3
                elif sum([x in playData for x in ["4th Quarter", "4th quarter", "2nd Half", "2nd half", " Game", " game"]]) > 0:
                    quarterNo = 4
                elif "OT" in playData:
                    quarterNo = "OT"
                else:
                    quarterNo = "UNKNOWN START QTR"
                    #raise ValueError("Could not determine quarter from {0}".format(playData))

        
        if gameclock is None and quarterNo is None:

            ## Check for specific format (HH:MM - 1st)
            m = re.match(r"\((.*?)\)", playData)
            if m is None:
                raise ValueError("Could not determine time and quarter from {0}".format(playData))

            try:
                text = m.group(0)
            except:
                raise ValueError("Could not extract regex result from {0}".format(playData))

               
            ## Check for overtime
            try:
                result = text[1:-1]
                if "OT" in result:
                    gameclock = timedelta(minutes=0, seconds=0) 
                    quarterNo = "OT"
            except:
                pass
            

            if gameclock is None:
                try:
                    result = text[1:-1]
                    result = result.split(" - ")
                    gameclock,quarter = result
                except:
                    raise ValueError("Could not parse time data: {0}".format(text))


                try:
                    minutes,seconds = gameclock.split(":")
                    gameclock = timedelta(minutes=int(minutes), seconds=int(seconds)) 
                except:
                    raise ValueError("Could not create timedelta for {0}".format(gameclock))


                quarterNo = {"1st":1, "2nd": 2, "3rd": 3, "4th": 4}.get(quarter)
                if quarterNo is None:
                    print("Could not find quarter from [{0}] or [{1}]".format(quarter, text))
                    #raise ValueError("Could not find quarter from [{0}] or [{1}]".format(quarter, text))


                try:
                    playData = playData.replace(text, "").strip()
                except:
                    raise ValueError("Could not remove [{0}] from [{1}]".format(text, playData))
                
        
        return gameclock,quarterNo,playData
    
    
    
        
        
    
    ########################################################################################################
    ##
    ## Determine Play Possession
    ##
    ########################################################################################################
    def determinePossession(self, play, players, debug=False):
        txt   = play.text
        
        ## Create empty object
        poss  = playpossessionclass(start=None, end=None, text=txt)
        

        ## Kickoff
        if isinstance(play, kickoffplay):
            poss.start = players.getKickingTeam(txt)
            if poss.start is None:
                poss.setUnknownStart()

        ## Field Goal
        if isinstance(play, fieldgoalplay):
            poss.start = players.getFGKickingTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()

        ## Punt
        if isinstance(play, puntplay):
            poss.start = players.getPuntingTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()

        ## Passing
        if isinstance(play, passingplay):
            poss.start = players.getPassingTeam(txt)
            if poss.start is None:
                poss.start = players.getReceivingTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()

        ## Sack
        if isinstance(play, sackplay):
            poss.start = players.getPassingTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()

        ## Running
        if isinstance(play, rushingplay):
            poss.start = players.getRunningTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()
            
        ## Safety
        if isinstance(play, safetyplay):
            if poss.start is None:
                poss.start = players.getPassingTeam(txt)
            if poss.start is None:
                poss.start = players.getRunningTeam(txt)
            if poss.start is None:
                poss.start = players.getPuntingTeam(txt)
            if poss.start is None:
                poss.setPreviousStart()

        ## Penalty/Timeout/End
        if isinstance(play, penaltyplay):
            poss.setPreviousStart()
            
        ## Empty play
        if isinstance(play, (endplay, timeoutplay)):
            poss.setPreviousStart()
            poss.valid=False
                     
        ## PAT
        if isinstance(play, patplay):
            poss.setPreviousStart()
                
        ############################################
        # Unclear...
        ############################################

        ## Touchdown
        if isinstance(play, touchdownplay):
            if poss.start is None:
                poss.start = players.getPassingTeam(txt)
            if poss.start is None:
                poss.start = players.getRunningTeam(txt)
            if poss.start is None:
                poss.start = players.getReceivingTeam(txt)
            if poss.start is None:
                poss.start = players.getPuntingTeam(txt)
            if poss.start is None:
                poss.start = players.getFGKickingTeam(txt)
            if poss.start is None:
                poss.start = players.getKickingTeam(txt)
            if poss.start is None:
                poss.setUnknownStart()

        ## Fumble (?)
        if isinstance(play, fumbleplay):
            poss.setUnknownStart()

        ## TBD (?)
        if isinstance(play, tbdplay):
            poss.setUnknownStart()
            
        return poss
        
    
    ########################################################################################################
    ##
    ## Analyze Possession For Every Play
    ##
    ########################################################################################################
    def analyzePossession(self, gameData, players):
        if not isinstance(gameData, list):
            raise ValueError("Game data is not a list!")
            
        for k,v in players.__dict__.items():
            print("{0: <10}{1}".format(k,v))
            
        unknown = False
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            prevStart  = None
            for ipl,drivePlay in enumerate(drivePlays):
                play       = drivePlay.play
                valid      = drivePlay.valid
                if valid is False:
                    continue
                possession = drivePlay.possession
                text       = possession.text
                status     = ""
                if possession.isUnknownStart():
                    unknown = True
                    status  = "<---- UNKNOWN"
                if possession.isPreviousStart():
                    if ipl == 0:
                        unknown = True
                        status  = "<---- UNKNOWN"
                    else:
                        try:
                            nextStart = drivePlays[ipl+1].possession.start
                        except:
                            nextStart = None
                        
                        if nextStart == prevStart:
                            possession.start = prevStart
                            status  = "<---- PREV/NXT"
                        else:
                            status  = "<---- PREV/UNK"
                prevStart = possession.start
                print("{0}\t{1}\t{2}\t{3: <15}{4: <15}{5}".format(idr,ipl,possession.start,play.name,status,text))

        if unknown is True:
            print("UNKNOWN PLAY START IN GAME")
            
        return gameData
        
    
    ########################################################################################################
    ##
    ## Combine all play data data together
    ##
    ########################################################################################################
    def parsePlayData(self, playStart, playData, fieldMap, players, debug=False):
        
        ##########################
        ## Find clock and quarter
        ##########################
        gameclock, quarter, playText = self.parseClockAndQuarter(playData)
        
        ##########################
        ## Determine play type
        ##########################
        ptype = playtype(playText)
        play  = ptype.getPlay()
        if ptype.valid is True:
            if play is None:
                raise ValueError("Could not determine play type from {0}".format(playText))
            play.analyze(debug=False)

        
        ##########################
        ## Check for valid play
        ##########################
        if ptype.valid is True and play is not None:
            validPlay = play.pa.getKey("noplay") != True
            if isinstance(play, (beginplay, endplay, timeoutplay)):
                validPlay = False
        else:
            validPlay = False
        
        
        
        ##########################
        ## Determine possession
        ##########################        
        if ptype.valid is True and play is not None:
            playPossession = self.determinePossession(play, players)
        else:
            playPossession = playpossessionclass(start=None, end=None)
            validPlay = False

        
        if playPossession.valid is False:
            validPlay = False
            
        
        ##########################
        ## Insert Clock/Quarter
        ##########################
        if playStart is None:
            playStart = playstartclass()
            
        if isinstance(playStart, playstartclass):
            playStart.setQuarter(quarter)
            playStart.setGameClock(gameclock)

            
        ### Result of play
        playResult = playclass(possession=playPossession, start=playStart, play=play, valid=validPlay)
        return playResult

        
    def parsePossession(self, possession, fieldMap, debug=False):
        if not isinstance(possession, list):
            raise ValueError("Possession is not a list: {0}".format(possession))
            
        if len(possession) != 1:
            raise ValueError("Not one element in possession list: {0}".format(possession))
            
        teamID = possession[0]
        
        teamAbbrev = None
        try:
            teamAbbrev = fieldMap[teamID]
        except:
            raise ValueError("Could not find {0} in field map: {1}".format(teamID, fieldMap))
            
        return teamAbbrev
        
        
    def augmentPlayStartWithPossession(self, possession, startVals, debug=False):
        if isinstance(startVals, playstartclass):
            
            ## Check for a team having the ball on their own side of the field (common)
            try:
                startY = int(startVals.startY)
            except:
                raise ValueError("Starting line {0} is not an integer".format(startVals.startY))

            if possession == startVals.side:
                distToEndZone = 100 - startY
            else:
                distToEndZone = startY                
                
            startVals.setStartY(startY)
            startVals.setDistToEndZone(distToEndZone)

        return startVals
    

    def parseHeadline(self, headline):
        pltype = None
        
        if isinstance(headline, list):
            if len(headline) >= 1:
                ptype  = playtype(headline[0])
                pltype = ptype.getPlay()
                
                if pltype is None:
                    raise ValueError("Could not determine headline play type: {0}".format(headline[0]))
            else:
                print("Not one headline entry: {0}".format(headline))
        else:
            raise ValueError("Headline is not a list: {0}".format(headline))
            
        return pltype
    
    
    def parseScore(self, score):
        if not isinstance(score, list):
            raise ValueError("Could not determine score type: {0}".format(score))
            
        if len(score) != 1:
            raise ValueError("Not one detail entry: {0}".format(score))
            
        scoredata = score[0]
        
        try:
            scoredata = int(scoredata)
        except:
            raise ValueError("Could not find an integer score for {0}".format(scoredata))
            
        return scoredata
    
    
    def parseDetail(self, detail):
        if not isinstance(detail, list):
            raise ValueError("Could not determine detail play type: {0}".format(detail))
            
        if len(detail) != 1:
            raise ValueError("Not one detail entry: {0}".format(detail))
            
        detaildata = detail[0]
        
        
        yards = "(yards|yard|Yds|yds|Yd|yd)"
        plays = "(play|plays)"        
        num  = "([+-?]\d+|\d+)"
        
        totalplays = None
        totalyards = None
        totalclock = None

        m = re.search(r"{0}\s{1},\s{2}\s{3},\s{4}:{5}".format(num, plays, num, yards, num, num), detaildata)
        if m is not None:
            groups = m.groups()
            totalplays = int(groups[0])
            totalyards = int(groups[2])
            totalclock = timedelta(minutes=int(groups[4]), seconds=int(groups[5]))

            
        if totalplays is None and totalyards is None and totalclock is None:
            m = re.search(r"{0}\s{1},\s{2}\s{3}".format(num, plays, num, yards), detaildata)
            if m is not None:
                groups = m.groups()
                totalplays = int(groups[0])
                totalyards = int(groups[2])
                totalclock = timedelta(minutes=0, seconds=0)
                
            
        if totalplays is None and totalyards is None and totalclock is None:
            raise ValueError("Could not parse drive detail: {0}".format(detaildata))

        drivedetail = drivedetailclass(plays=totalplays, yards=totalyards, gametime=totalclock)
            
        return drivedetail
    
    
    
    ################################################################################################################################################################
    ## Change of Possession Plays
    ################################################################################################################################################################
    def splitChangeOfPossession(self, gameData, gameStatsData, copMap, debug=False):
        gameResults = self.splitKickoff(gameData, gameStatsData, copMap, debug=False)
        gameResults = self.splitPunt(gameData, gameStatsData, copMap, debug=False)
        gameResults = self.splitInterception(gameData, gameStatsData, copMap, debug=False)
        
        for idr,driveData in enumerate(gameData):
            drivePlays = driveData.plays
            for ipl,drivePlay in enumerate(drivePlays):
                play       = drivePlay.play
                valid      = drivePlay.valid
                if valid is False:
                    continue
                possession = drivePlay.possession
                text       = possession.text
                status     = ""
                print("{0}\t{1}\t{2}\t{3: <15}{4}".format(idr,ipl,possession.start,play.name,text))
            print("")
        return gameResults
    
        
    ######################################################
    ## Split play into [kick] + [return + X]
    ######################################################
    def splitKickoff(self, gameData, gameStatsData, copMap, debug=False):
        for idr in range(len(gameData)):
            drivePlays  = gameData[idr].plays
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.valid is not True:
                    continue
                if isinstance(playData.play, kickoffplay):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.kickoffplay = False
                    newReturnPlay.pa.touchdown   = playData.play.pa.touchdown
                    newReturnPlay.pa.fumble      = playData.play.pa.fumble
                    newReturnPlay.pa.runback     = playData.play.pa.runback
                    playData.play.pa.touchdown   = False
                    playData.play.pa.fumble      = False
                    playData.play.pa.runback     = False
                    newPossession = playpossessionclass(start=None, end=None, text=playData.play.text)
                    try:
                        newPossession.start = copMap[playData.possession.start]
                    except:
                        newPossession.start = playData.possession.setUnknownStart()
                    newPlayData = playclass(possession=newPossession, start=playData.start, play=newReturnPlay, valid=playData.valid)
                    drivePlays.insert(dp+1, newPlayData)
                    break
    
        
    ######################################################
    ## Split play into [punt] + [return + X]
    ######################################################
    def splitPunt(self, gameData, gameStatsData, copMap, debug=False):
        for idr in range(len(gameData)):
            drivePlays  = gameData[idr].plays
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.valid is not True:
                    continue
                if isinstance(playData.play, puntplay):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.puntplay = False
                    newReturnPlay.pa.touchdown   = playData.play.pa.touchdown
                    newReturnPlay.pa.fumble      = playData.play.pa.fumble
                    newReturnPlay.pa.runback     = playData.play.pa.runback
                    playData.play.pa.touchdown   = False
                    playData.play.pa.fumble      = False
                    playData.play.pa.runback     = False
                    newPossession = playpossessionclass(start=None, end=None, text=playData.play.text)
                    try:
                        newPossession.start = copMap[playData.possession.start]
                    except:
                        newPossession.start = playData.possession.setUnknownStart()
                    newPlayData = playclass(possession=newPossession, start=playData.start, play=newReturnPlay, valid=playData.valid)
                    drivePlays.insert(dp+1, newPlayData)
                    break
    
        
    ######################################################
    ## Split play into [interception] + [return + X]
    ######################################################
    def splitInterception(self, gameData, gameStatsData, copMap, debug=False):
        for idr in range(len(gameData)):
            drivePlays  = gameData[idr].plays
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.valid is not True:
                    continue
                if isinstance(playData.play, passingplay) and playData.play.pa.getKey("interception"):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.passingplay = False
                    newReturnPlay.pa.touchdown   = playData.play.pa.touchdown
                    newReturnPlay.pa.fumble      = playData.play.pa.fumble
                    newReturnPlay.pa.runback     = playData.play.pa.runback
                    playData.play.pa.touchdown   = False
                    playData.play.pa.fumble      = False
                    playData.play.pa.runback     = False
                    newPossession = playpossessionclass(start=None, end=None, text=playData.play.text)
                    try:
                        newPossession.start = copMap[playData.possession.start]
                    except:
                        newPossession.start = playData.possession.setUnknownStart()
                    newPlayData = playclass(possession=newPossession, start=playData.start, play=newReturnPlay, valid=playData.valid)
                    drivePlays.insert(dp+1, newPlayData)
                    break
        
    
    def parseGames(self, gameID=None, debug=False):
        if hist is None:
            raise ValueError("Must set historical class!")

        self.unknownPlays = []
        
        self.poorlyParsed = ['401012731', '400547781', '400547808']
            
        files    = findExt(hist.getGamesResultsDir(), ext=".p", debug=False)
        for ifile in files:
            print(ifile)
            try:
                year = int(getBaseFilename(ifile).split("-")[0])
            except:
                raise ValueError("Could not get year from {0}".format(ifile))
            
            if year < 2014:
                continue
                
            yearData = getFile(ifile)
            
            statsFilename = setFile(hist.getStatisticsResultsDir(), "{0}-stats.json".format(year))
            statsData = getFile(statsFilename)

            if len(self.unknownPlays) > 75:
                break


            
            totalGames = 0
            for gameIdent,gameData in yearData.items():
                if gameID is not None:
                    if gameID != gameIdent:
                        continue
                
                teamsMetaData    = gameData["Teams"]
                homeTeamMetaData = teamsMetaData["Home"]
                awayTeamMetaData = teamsMetaData["Away"]
                driveData        = gameData["Plays"]
                
                
                print("GAME ID: {0}".format(gameIdent))
                if debug:
                    print("  Home Team: {0}".format(homeTeamMetaData))
                    print("  Away Team: {0}".format(awayTeamMetaData))

                fieldMap = {}
                fieldMap[homeTeamMetaData["ID"]]     = homeTeamMetaData["Abbrev"]
                fieldMap[homeTeamMetaData["Abbrev"]] = homeTeamMetaData["ID"]
                fieldMap[awayTeamMetaData["ID"]]     = awayTeamMetaData["Abbrev"]
                fieldMap[awayTeamMetaData["Abbrev"]] = awayTeamMetaData["ID"]
                
                fieldMap["Home"] = homeTeamMetaData["Abbrev"]
                fieldMap["Away"] = awayTeamMetaData["Abbrev"]
                
                copmap = {}
                copmap[homeTeamMetaData["Abbrev"]] = awayTeamMetaData["Abbrev"]
                copmap[awayTeamMetaData["Abbrev"]] = homeTeamMetaData["Abbrev"]



                if gameIdent in hist.noGameData:
                    continue
                    
                if gameIdent in self.poorlyParsed:
                    continue
                                
                
                ################################################################################
                ### Learn key plays for use in determining possession
                ################################################################################
                gameStatsData = gameplayers(teamsMap=fieldMap, statsData=statsData)
                gameStatsData.addPunters(driveData, debug=False)
                
                if False:
                    saveFile(idata=driveData, ifile="testDriveData.p", debug=True)
                    saveFile(idata=fieldMap,  ifile="testFieldMap.p",  debug=True)
                    saveFile(idata=gameStatsData,  ifile="testGameStats.p", debug=True)
                    1/0

                if len(self.unknownPlays) > 75:
                    break
                    
                
                gameResult = []
                totalPlays = 0
                
                
                ################################################################################
                ### Iterate over drives
                ################################################################################
                for idr,drive in enumerate(driveData):
                    ### Fix known drive problems
                    #drive = self.fixDrive(gameIdent, idr, drive)
                    
                    driveNo    = drive['Drive']
                    headline   = drive['Headline']
                    detail     = drive['Detail']
                    ### For whatever reason home/away scores are reversed on the webpage...
                    homescore  = drive['AwayScore']
                    awayscore  = drive['HomeScore']
                    possession = drive['Posession'] ## note this needs to change later
                    drivedata  = drive['Data']
                    
                    
                    
                    if debug:
                        print("Drive: {0} \t{1} (H) - {2} (A)".format(idr,homescore, awayscore))
                    
    
                    ## Determine possession as defined in the play start fields later
                    possession = self.parsePossession(possession, fieldMap)
                    headline   = self.parseHeadline(headline)
                    detail     = self.parseDetail(detail)
                    homescore  = self.parseScore(homescore)
                    awayscore  = self.parseScore(awayscore)
            
        
                    fullDriveData = driveclass(headline=headline, detail=detail, possession=possession,
                                              postdrivehomescore=homescore, postdriveawayscore=awayscore)
                    
                    drivePlays = []
            
                
                    ################################################################################
                    ### Iterate over plays in drive
                    ################################################################################
                    for ipl,play in enumerate(drivedata):
                        playNo        = play['Play']
                        playStartText = play['Start']
                        playData      = play['Data']

                        ### Determine play starting position and clock
                        startVals = self.parsePlayStart(playStartText, playNo, len(drivedata), playData)
                        startVals = self.augmentPlayStartWithPossession(possession, startVals)
                        
                        ### Determine type of play
                        playResult = self.parsePlayData(startVals, playData, fieldMap, gameStatsData)
                        
                        
                        if playResult.possession.start is None and playResult.valid is True:
                            raise ValueError("ERROR WITH POSSESSION:",playResult.play.name,'\t',playResult.play.text)
                        
                        totalPlays += 1
                        drivePlays.append(playResult)

                    
                    fullDriveData.setPlays(drivePlays)
                    #fullDriveData = self.insertMissingData(gameIdent, idr, fullDriveData, debug)
                    #fullDriveData = self.augmentPlayWithScore(fullDriveData, fieldMap, debug)
                    gameResult.append(fullDriveData)


                
                   
                ################################################################################
                ### Analyze Possession
                ################################################################################ 
                gameResult = self.analyzePossession(gameResult, gameStatsData)
                gameResult = self.splitChangeOfPossession(gameResult, gameStatsData, copmap, debug=True)
                1/0
                
                continue
                
                print("Found a total of {0} drives in this game".format(len(gameResult)))
                gameResult  = self.splitChangeOfPossession(gameResult, fieldMap, debug=True)
                scoreResult = self.analyzeGameScore(gameResult, fieldMap, debug=True)
                self.analyzeGameData(gameResult)
                if scoreResult is False:
                    raise ValueError("There was a problem parsing the scores for this game {0}!".format(gameIdent))


                    
                totalGames += 1
                if debug:
                    print("Found {0} plays in this game {1}".format(totalPlays, gameID))
                    
                    
            print("Found {0} total games for {1}".format(totalGames, ifile))
                
_ = clock("Last Run")

Current Time is Thu Apr 04, 2019 22:17:22 for Last Run


In [269]:
pbp = playbyplay()
pbp.setHistorical(hist)
pbp.parseGames(gameID=None, debug=False)

/Volumes/Blue/Football/results/games/2003-games.p
/Volumes/Blue/Football/results/games/2004-games.p
/Volumes/Blue/Football/results/games/2005-games.p
/Volumes/Blue/Football/results/games/2006-games.p
/Volumes/Blue/Football/results/games/2007-games.p
/Volumes/Blue/Football/results/games/2008-games.p
/Volumes/Blue/Football/results/games/2009-games.p
/Volumes/Blue/Football/results/games/2010-games.p
/Volumes/Blue/Football/results/games/2011-games.p
/Volumes/Blue/Football/results/games/2012-games.p
/Volumes/Blue/Football/results/games/2013-games.p
/Volumes/Blue/Football/results/games/2014-games.p
GAME ID: 400547640
fieldMap  {'238': 'VAN', 'VAN': '238', '218': 'TEM', 'TEM': '218', 'Home': 'VAN', 'Away': 'TEM'}
homeTeamPlayers<__main__.teamplayers object at 0x1a1c651d30>
homeTeamNameVAN
awayTeamPlayers<__main__.teamplayers object at 0x1a25459278>
awayTeamNameTEM
punters   {'Alex Starzyk': Counter({'TEM': 5, 'VAN': 2}), 'Colby Cooke': Counter({'VAN': 5, 'TEM': 2})}
possessionTEM
0	0	VAN	kick

ZeroDivisionError: division by zero

In [ ]:
x = 1.33454655
sig = 4

# Test Area

In [231]:
year = 2014
driveData = getFile(ifile="testDriveData.p", debug=True)
fieldMap  = getFile(ifile="testFieldMap.p",  debug=True)
gameStatsData = getFile(ifile="testGameStats.p",  debug=True)
fieldMap

Loading data from testDriveData.p
  --> This file is 8.7kB.
Loading testDriveData.p
Loading data from testFieldMap.p
  --> This file is 74B.
Loading testFieldMap.p
Loading data from testGameStats.p
  --> This file is 8.9kB.
Loading testGameStats.p


{'238': 'VAN',
 'VAN': '238',
 '218': 'TEM',
 'TEM': '218',
 'Home': 'VAN',
 'Away': 'TEM'}

In [232]:
pbp = playbyplay()
pbp.setHistorical(hist)


################################################################################
### Iterate over drives
################################################################################
debug=True
for idr,drive in enumerate(driveData):
    drivedata  = drive['Data']
    ################################################################################
    ### Iterate over plays in drive
    ################################################################################
    for ipl,play in enumerate(drivedata):
        playNo        = play['Play']
        playStartText = play['Start']
        playData      = play['Data']

        startVals  = pbp.parsePlayStart(playStartText, playNo, len(drivedata), playData)
        startVals  = pbp.augmentPlayStartWithPossession(possession, startVals)
        playResult = pbp.parsePlayData(startVals, playData, fieldMap, gameStatsData)

        print("{0: <4}{1: <4}{2: <6}{3}".format(idr,ipl,playResult.possession.start,playData))


  Using Kicking Text: Hayden Lekacz
  EXACT MATCH: PK <-- Hayden Lekacz
  Kicking Team: VAN
0   0   VAN   (14:55 - 1st) Hayden Lekacz kickoff for 64 yds , Khalif Herbin return for 18 yds to the Temp 19
0   1   PREV  (14:55 - 1st) TEMPLE Penalty, False Start (5 Yards) to the Temp 14
0   2   TEM   (14:45 - 1st) P.J. Walker pass complete to Jalen Fitzpatrick for 3 yds to the Temp 17
0   3   TEM   (14:20 - 1st) Kenneth Harper run for 5 yds to the Temp 22
0   4   TEM   (13:50 - 1st) P.J. Walker pass complete to Jalen Fitzpatrick for 8 yds to the Temp 30 for a 1ST down
0   5   TEM   (13:25 - 1st) P.J. Walker pass complete to Nate Hairston for 2 yds to the Temp 32
0   6   TEM   (12:58 - 1st) P.J. Walker pass incomplete to Brandon Shippen
0   7   TEM   (12:50 - 1st) Kenneth Harper run for 5 yds to the Temp 37
0   8   TEM   (12:07 - 1st) Alex Starzyk punt for 43 yds , Darrius Sims returns for 8 yds to the Vandy 28
1   0   VAN   (12:00 - 1st) Patton Robinette run for 3 yds to the Vandy 31
1   1 

In [214]:
################################################################################
### Learn key plays for use in determining possession
################################################################################
gp = gameplayers(teamsMap=fieldMap, statsData=statsData)
print(gp.getPassingTeam("Phillip Walker passes for 2 yards"))
print(gp.getPassingTeam("P.J. Walker passes for 2 yards", debug=False))
print(gp.getRunningTeam("Ralph Webb runs for 2 yards", debug=False))
print(gp.getFGKickingTeam("Austin Jones 47 Yd FG", debug=False))
print(gp.getKickingTeam("Austin Jones Kickoff for touchback", debug=False))
print(gp.getPuntingTeam("Austin Jones punt for 35 yards", debug=False))
print(gp.getPuntingTeam("A. Jones punt for 35 yards", debug=False))
print(gp.getPuntingTeam("Alex Starzyk punt for 43 yds , Darrius Sims returns for 8 yds to the Vandy 28"))
print(gp.getReceivingTeam("Phillip Walker pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.getReceivingTeam("Johnny McCrary pass complete to C.J. Duncan for 6 yds", debug=False))
print(gp.getReceivingTeam("Austin Jones pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.getPassingTeam("Austin Jones pass complete to Jalen Fitzpatrick for 6 yds", debug=False))
print(gp.homeTeamPlayers.punters)
print(gp.awayTeamPlayers.punters)

TEM
TEM
VAN
TEM
TEM
TEM
TEM
None
TEM
VAN
TEM
None
{'Tommy Openshaw': 'PK', 'T. Openshaw': 'PK', 'Hayden Lekacz': 'PK', 'H. Lekacz': 'PK'}
{'Austin Jones': 'PK', 'A. Jones': 'PK', 'Tyler Mayes': 'P', 'T. Mayes': 'P'}


In [224]:
from searchUtils import findNearest
from collections import Counter

class teamplayers:
    def __init__(self, statsData):
        if statsData is None:
            raise ValueError("Stats data is empty!")
            
        self.statsData  = statsData
        self.kickers    = {}
        self.fgkickers  = {}
        self.punters    = {}
        self.passers    = {}
        self.runners    = {}
        self.receivers  = {}
        self.defense    = {}

        self.addPlayers()

    def getShortName(self, name):
        vals  = name.split()
        start = ["{0}.".format(y[0]) for y in vals[:-1]]
        end   = vals[-1]
        sname = " ".join([" ".join(start), end])
        return sname
                

    def addPlayers(self):
        ## Get Kickers
        try:
            kickingData = self.statsData['Kicking']
            for kicker in kickingData.keys():
                name,pos = kicker.split("$")
                if name == "TOTAL" or pos == "ALL":
                    continue
                self.kickers[name]                      = pos
                self.kickers[self.getShortName(name)]   = pos
                
                self.fgkickers[name]                    = pos
                self.fgkickers[self.getShortName(name)] = pos

                self.punters[name]                      = pos
                self.punters[self.getShortName(name)]   = pos
        except:
            print("Could not set kicker data!")
        
        
        ## Get Passers
        try:
            passingData = self.statsData['Passing']
            for passer in passingData.keys():
                name,pos = passer.split("$")
                if name == "TOTAL" or pos == "ALL":
                    continue
                self.passers[name]                      = pos
                self.passers[self.getShortName(name)]   = pos
        except:
            print("Could not set passing data!")
        
        
        ## Get Rushers
        try:
            rushingData = self.statsData['Rushing']
            for rusher in rushingData.keys():
                name,pos = rusher.split("$")
                if name == "TOTAL" or pos == "ALL":
                    continue
                self.runners[name]                      = pos
                self.runners[self.getShortName(name)]   = pos
        except:
            print("Could not set rushing data!")
        
        
        ## Get Receivers
        try:
            receivingData = self.statsData['Receiving']
            for receiver in receivingData.keys():
                name,pos = receiver.split("$")
                if name == "TOTAL" or pos == "ALL":
                    continue
                self.receivers[name]                      = pos
                self.receivers[self.getShortName(name)]   = pos
        except:
            print("Could not set receiving data!")
        
        
        ## Get Defense
        try:
            defenseData = self.statsData['Defense']
            for defense in defenseData.keys():
                name,pos = defense.split("$")
                self.defense[name]                      = pos
                self.defense[self.getShortName(name)]   = pos
        except:
            print("Could not set defense data!")

    
    
class gameplayers:
    def __init__(self, statsData, teamsMap):
        self.fieldMap = teamsMap
        
        homeTeam   = teamsMap['Home']
        homeTeamID = teamsMap[homeTeam]
        awayTeam   = teamsMap['Away']
        awayTeamID = teamsMap[awayTeam]

        homeTeamStatsData = statsData.get(homeTeamID)
        if homeTeamStatsData is None:
            print("WARNING: Home Team {0} stats data is NONE".format(homeTeam))
        awayTeamStatsData = statsData.get(awayTeamID)
        if awayTeamStatsData is None:
            print("WARNING: Away Team {0} stats data is NONE".format(homeTeam))
            
        self.homeTeamPlayers = teamplayers(homeTeamStatsData)
        self.homeTeamName    = homeTeam
        self.awayTeamPlayers = teamplayers(awayTeamStatsData)
        self.awayTeamName    = awayTeam
        
    
        self.punters   = {}
        

    ### Punters aren't included so we have to include them differently
    def addPunters(self, driveData, debug=False):
        for idr,drive in enumerate(driveData):
            self.possession = drive['Posession'][0]
            if self.fieldMap is not None:
                try:
                    self.possession = self.fieldMap[self.possession]
                except:
                    raise ValueError("ERROR with field map: {0}, {1}".format(self.fieldMap, self.possession))
                    
            plays  = drive['Data']
            for ipl,play in enumerate(plays):
                playText      = play['Data']
                if debug:
                    print(idr,'\t',ipl,'\t',playText)
                self.analyzeText(playText, debug)

            if debug:
                print("")
                
        for name,punterData in self.punters.items():
            team = punterData.most_common(1)[0][0]
            if team == self.homeTeamName:
                if debug:
                    print("Adding {0} to {1}".format(name, self.homeTeamName))
                self.homeTeamPlayers.punters[name] = "P"
            elif team == self.awayTeamName:
                if debug:
                    print("Adding {0} to {1}".format(name, self.awayTeamName))
                self.awayTeamPlayers.punters[name] = "P"
            else:
                raise ValueError("Could not add punter to any team!")

                
    def analyzeText(self, playText, debug=False):
        ## Remove clock/quarter information
        pos = playText.find(")")
        if pos > 0 and pos < len(playText) - 2:
            playText = playText[pos+1:].strip()
            
        keys = [" punt for ", " Punt For ", " PUNT FOR ", " punts for ", " Punts For ", " PUNTS FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            text = playText[:pos].strip()
            if self.punters.get(text) is None:
                self.punters[text] = Counter()
            self.punters[text][self.possession] += 1


    def findPlayerTeam(self, playText, playersDict, useNearest=0.8, debug=False):
        for name,pos in playersDict.items():
            if name in playText:
                if debug:
                    print("  EXACT MATCH: {0} <-- {1}".format(pos, name))
                return pos
        
        if useNearest is not None:
            pos = findNearest(item=playText, ilist=playersDict.keys(), cutoff=useNearest, num=1)
            if len(pos) == 0:
                return None
            else:
                name = pos[0]
                pos  = playersDict[name]
                if debug:
                    print("  NEAR MATCH: {0} <-- {1}".format(name, playText))
                return pos
        
        return None
    
    
    def findReturnValue(self, posnames, posH, posA):        
        if posH is not None and posA is None:
            return self.homeTeamName
        elif posA is not None and posH is None:
            return self.awayTeamName
        elif posH is None and posA is None:
            return None
        else:
            if posH in posnames:
                return self.homeTeamName
            elif posA in posnames:
                return self.awayTeamName

        return None

        
        
    ################################################################################################
    # Kicking Players
    ################################################################################################
    def getKickingTeam(self, playText, debug=False):
        posnames = ["K", "PK", "P"]
        
        ## Fix Text
        keys = ["kickoff", "Kickoff", "KICKOFF"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            text = playText[:pos].strip()
        else:
            text = playText
            
        if debug:
            print("  Using Kicking Text: {0}".format(text))
                        
        posH   = self.findPlayerTeam(text, self.homeTeamPlayers.kickers, debug=debug)
        posA   = self.findPlayerTeam(text, self.awayTeamPlayers.kickers, debug=debug)        
        retval = self.findReturnValue(posnames, posH, posA)
        if debug:
            print("  Kicking Team: {0}".format(retval))
        return retval
    
        
        
    ################################################################################################
    # Field Goal Players
    ################################################################################################
    def getFGKickingTeam(self, playText, debug=False):
        posnames = ["PK"]
        retval = None
        
        ## Fix Text        
        if playText[-1] == ")":
            pos = playText.rfind("(")
            if pos > 0:
                txt = playText[pos+1:-1]
                keys = [" kick", " Kick", " KICK"]
                pos = max([txt.find(x) for x in keys])
                if pos > 0:
                    text = txt[:pos].strip()
                        
                    posH   = self.findPlayerTeam(text, self.homeTeamPlayers.fgkickers, debug=debug)
                    posA   = self.findPlayerTeam(text, self.awayTeamPlayers.fgkickers, debug=debug)        
                    retval = self.findReturnValue(posnames, posH, posA)
              
        if retval is None:
            kick = ("(fg|FG)")
            num  = "([+-?]\d+|\d+)"  
            dist = ("(yards|yard|Yds|yds|Yd|yd)")

            m = re.split("{0}\s{1}\s{2}".format(num, dist, kick), playText)
            if len(m) > 1:
                text = m[0].strip()
                        
                posH   = self.findPlayerTeam(text, self.homeTeamPlayers.fgkickers, debug=debug)
                posA   = self.findPlayerTeam(text, self.awayTeamPlayers.fgkickers, debug=debug)        
                retval = self.findReturnValue(posnames, posH, posA)
              
        if retval is None:
            wrd1 = ("(field|Field|FIELD)")
            wrd2 = ("(goal|Goal|GOAL)")
            num  = "([+-?]\d+|\d+)"  
            dist = ("(yards|yard|Yard|Yds|yds|Yd|yd)")

            m = re.split("{0}\s{1}\s{2}\s{3}".format(num, dist, wrd1, wrd2), playText)
            if len(m) > 1:
                text = m[0].strip()
                        
                posH   = self.findPlayerTeam(text, self.homeTeamPlayers.fgkickers, debug=debug)
                posA   = self.findPlayerTeam(text, self.awayTeamPlayers.fgkickers, debug=debug)        
                retval = self.findReturnValue(posnames, posH, posA)
        
        if debug:
            print("  Field Goal Kicking Team: {0}".format(retval))
        return retval
    
        
        
    ################################################################################################
    # Punting Players
    ################################################################################################
    def getPuntingTeam(self, playText, debug=False):
        posnames = ["P", "PK", "K"]
        retval = None
        
        ## Fix Text        
        keys = [" punt for ", " Punt For ", " PUNT FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            text = playText[:pos].strip()
        else:
            text = playText

        posH   = self.findPlayerTeam(text, self.homeTeamPlayers.punters, debug=debug)
        posA   = self.findPlayerTeam(text, self.awayTeamPlayers.punters, debug=debug)        
        retval = self.findReturnValue(posnames, posH, posA)
        
        if debug:
            print("  Punting Team: {0}".format(retval))        
        return retval
    
    
        
    ################################################################################################
    # Passing Players
    ################################################################################################
    def getPassingTeam(self, playText, debug=False):
        posnames = ["QB"]
        
        ## Fix Text
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE", "passes"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            text = playText[:pos].strip()
        else:
            text = playText
            
        if debug:
            print("  Using Passing Text: {0}".format(text))
                        
        posH   = self.findPlayerTeam(text, self.homeTeamPlayers.passers, debug=debug)
        posA   = self.findPlayerTeam(text, self.awayTeamPlayers.passers, debug=debug)        
        retval = self.findReturnValue(posnames, posH, posA)
        
        if debug:
            print("  Passing Team: {0}".format(retval))
        return retval
    
        
        
    ################################################################################################
    # Running Players
    ################################################################################################
    def getRunningTeam(self, playText, debug=False):
        posnames = ["RB", "QB", "WR"]
        retval = None
        
        ## Fix Text
        keys = [" run for ", " Run For ", " RUN FOR ", " runs for ", " Runs For ", " RUNS FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            text = playText[:pos].strip()
        else:
            text = playText
            
        if debug:
            print("  Using Running Text: {0}".format(text))
                        
        posH   = self.findPlayerTeam(text, self.homeTeamPlayers.runners, debug=debug)
        posA   = self.findPlayerTeam(text, self.awayTeamPlayers.runners, debug=debug)        
        retval = self.findReturnValue(posnames, posH, posA)
        
        if retval is None:
            run  = ("(run|Run|RUN)")
            num  = "([+-?]\d+|\d+)"  
            dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
            m = re.split("{0}\s{1}\s{2}".format(num, dist, run), playText)
            if len(m) > 1:
                text = m[0].strip()          

                if debug:
                    print("  Using Running Text: {0}".format(text))

                posH   = self.findPlayerTeam(text, self.homeTeamPlayers.runners, debug=debug)
                posA   = self.findPlayerTeam(text, self.awayTeamPlayers.runners, debug=debug)        
                retval = self.findReturnValue(posnames, posH, posA)          
        
        if debug:
            print("  Running Team: {0}".format(retval))
        return retval
    
        
        
    ################################################################################################
    # Receiving Players
    ################################################################################################
    def getReceivingTeam(self, playText, debug=False):
        posnames = ["WR", "RB"]
        retval = None
        
        ## Fix Text
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            txt = playText[pos:].strip()
            txt = " ".join(txt.split()[2:])[3:].strip()
            keys = [" for ", " For ", " FOR "]
            pos = [x for x in [txt.find(x) for x in keys] if x != -1]
            if len(pos) > 0 and min(pos) > 0:
                text = txt[:min(pos)].strip()

                posH   = self.findPlayerTeam(text, self.homeTeamPlayers.receivers, debug=debug)
                posA   = self.findPlayerTeam(text, self.awayTeamPlayers.receivers, debug=debug)        
                retval = self.findReturnValue(posnames, posH, posA)          
        
        if debug:
            print("  Receiving Team: {0}".format(retval))
        return retval
                
        
        
    ################################################################################################
    # Defense Players
    ################################################################################################
    def getDefenseTeam(self, playText, debug=False):
        return None
        pos = self.findPlayerTeam(playText, self.homeTeamPlayers.defense)
        if pos is not None:
            return self.homeTeamName

        pos = self.findPlayerTeam(playText, self.awayTeamPlayers.defense)
        if pos is not None:
            return self.awayTeamName
        
        return None

In [144]:
from searchUtils import findNearest
x1 = ['Johnny McCrary', 'Patton Robinette', 'Wade Freebeck', 'Stephen Rivers', 'Josh Grady']
x2 = ['Phillip Walker', 'P. Walker', 'Connor Reilly', 'Jalen Fitzpatrick', 'Tim DiGiorgio']
x  = "P.J. Walker"
print(findNearest(cutoff=0.7, ilist=x1, item=x, num=1))
print(findNearest(cutoff=0.9, ilist=x2, item=x, num=1))

[]
['P. Walker']


In [ ]:
from collections import Counter

class positionplayers:
    def __init__(self):
        self.kickers   = None
        self.fgkickers = None
        self.punters   = None
        self.runners   = None
        self.receivers = None
        self.passers   = None
        self.defense   = None
        
    
        
    def getKickingTeam(self, playText):
        if isinstance(self.kickers, dict):
            for player, team in self.kickers.items():
                if player in playText:
                    return team
        return None
        
        
    def getFGKickingTeam(self, playText):
        if isinstance(self.fgkickers, dict):
            for player, team in self.fgkickers.items():
                if player in playText:
                    return team
        return None
        
        
    def getPuntingTeam(self, playText):
        if isinstance(self.punters, dict):
            for player, team in self.punters.items():
                if player in playText:
                    return team
        return None
        
        
    def getRunningTeam(self, playText):
            
        if isinstance(self.runners, dict):
            for player, team in self.runners.items():
                if player in playText:
                    return team
                

        run  = ("(run|Run|RUN)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")

        m = re.split("{0}\s{1}\s{2}".format(num, dist, run), playText)
        if len(m) > 1:
            runner = m[0].strip()
                
                
        return None
        
        
    def getPassingTeam(self, playText):
        ## Fix Text
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
        else:
            passer = playText
        
        if isinstance(self.passers, dict):
            for player, team in self.passers.items():
                if player in playText:
                    return team
        return None
        
        
    def getReceivingTeam(self, playText):
        if isinstance(self.receivers, dict):
            for player, team in self.receivers.items():
                if player in playText:
                    return team
        return None
    

class keyplayers:
    def __init__(self, fieldMap=None, driveData=None):
        self.driveData = driveData
        
        self.teams     = {}
        self.kickers   = Counter()
        self.fgkickers = Counter()
        self.punters   = Counter()
        self.runners   = Counter()
        self.receivers = Counter()
        self.passers   = Counter()
        self.defense   = Counter()
        
        self.fieldMap  = fieldMap
        self.teamsmap  = {}
        
        self.possession = None
        
        
    def getPlayers(self, lookup=None):        
        players = positionplayers()
        
        if len(self.teamsmap) != 2:
            raise ValueError("There must be two teams!: [{0}]".format(self.teamsmap))
        keys = list(self.teamsmap.keys())
        
        if lookup is not None:
            self.teamsmap[keys[0]] = lookup[keys[1]]
            self.teamsmap[keys[1]] = lookup[keys[0]]
            
            self.lookup = lookup
        else:
            self.teamsmap[keys[0]] = keys[1]
            self.teamsmap[keys[1]] = keys[0]
            
            self.lookup = {}
            self.lookup[keys[0]] = keys[0]
            self.lookup[keys[1]] = keys[1]
        
        
        print(self.passers)
        
        passerNames = [x[0] for x in self.passers.most_common()]
        passerTeams = [self.teams[x].most_common(1)[0][0] for x in passerNames]
        print(passerTeams)
        passerTeams = [self.lookup[x] for x in passerTeams]
        players.passers     = dict(zip(passerNames, passerTeams))
        print(players.passers)
        
        fgkickerNames = [x[0] for x in self.fgkickers.most_common()]
        fgkickerTeams = [self.teams[x].most_common(1)[0][0] for x in fgkickerNames]
        fgkickerTeams = [self.lookup[x] for x in fgkickerTeams]
        players.fgkickers     = dict(zip(fgkickerNames, fgkickerTeams))

        ### I need to swap the kickers because they show up with the other team in the text
        kickerNames = [x[0] for x in self.kickers.most_common()]
        kickerTeams = [self.teams[x].most_common(1)[0][0] for x in kickerNames]
        kickerTeams = [self.teamsmap[x] for x in kickerTeams]
        players.kickers     = dict(zip(kickerNames, kickerTeams))

        runnerNames = [x[0] for x in self.runners.most_common()]
        runnerTeams = [self.teams[x].most_common(1)[0][0] for x in runnerNames]
        runnerTeams = [self.lookup[x] for x in runnerTeams]
        players.runners     = dict(zip(runnerNames, runnerTeams))

        receiverNames = [x[0] for x in self.receivers.most_common()]
        receiverTeams = [self.teams[x].most_common(1)[0][0] for x in receiverNames]
        receiverTeams = [self.lookup[x] for x in receiverTeams]
        players.receivers     = dict(zip(receiverNames, receiverTeams))

        punterNames = [x[0] for x in self.punters.most_common()]
        punterTeams = [self.teams[x].most_common(1)[0][0] for x in punterNames]
        punterTeams = [self.lookup[x] for x in punterTeams]
        players.punters     = dict(zip(punterNames, punterTeams))

        defenseNames = [x[0] for x in self.defense.most_common()]
        defenseTeams = [self.teams[x].most_common(1)[0][0] for x in defenseNames]
        defenseTeams = [self.lookup[x] for x in defenseTeams]
        players.defenses     = dict(zip(defenseNames, defenseTeams))
    
        return players


        
    def setDriveData(self, driveData):
        self.driveData = driveData
    
    def setPlayText(self, playText):
        self.playText = playText

        
    ################################################################################
    ## Analyze Game
    ################################################################################
    def analyzeGame(self, debug=False):
        if self.driveData is None:
            print("No Game Data")
            return
    
        for idr,drive in enumerate(self.driveData):
            self.possession = drive['Posession'][0]
            if self.fieldMap is not None:
                try:
                    self.possession = self.fieldMap[self.possession]
                except:
                    raise ValueError("ERROR with field map: {0}, {1}".format(self.fieldMap, self.possession))
            self.teamsmap[self.possession] = None
            drivedata  = drive['Data']

            for ipl,play in enumerate(drivedata):
                playText      = play['Data']
                if debug:
                    print(idr,'\t',ipl,'\t',playText)
                self.analyzeText(playText, debug)

            if debug:
                print("")
                
    def analyzeText(self, playText, debug=False):
        ## Remove clock/quarter information
        pos = playText.find(")")
        if pos > 0 and pos < len(playText) - 2:
            playText = playText[pos+1:].strip()
            
        self.addPasser(playText, debug)
        self.addRunner(playText, debug)
        self.addReceiver(playText, debug)
        self.addKicker(playText, debug)
        self.addFGKicker(playText, debug)
        self.addPunt(playText, debug)
        self.addDefense(playText, debug)

        
    def showAdd(self, name, value, possession, text, debug=False):
        if debug:
            sumval = sum(self.teams[name][value].values())
            indval = self.teams[name][value].most_common(1)[0][1]
            frac   = round(100*indval/sumval,0)
            
            print("\t{0: <15}{1: <30} --> {2: <10}  [{3}, {4}/{5}] \t [{6}]".format(name, value, self.possession, frac, indval, sumval, text))
            
                           

    ################################################################################
    ## Add Player <-> Team
    ################################################################################
    def addPlayerTeam(self, name, value):
        if self.teams.get(name) is None:
            self.teams[name] = {}
            
        if self.teams[name].get(value) is None:
                self.teams[name][value] = Counter()
        self.teams[name][value][self.possession] += 1                           
                           
            
    ################################################################################
    ## Passers
    ################################################################################
    def addPasser(self, playText, debug=False):
        name = "passer"
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
            self.passers[passer] += 1
            self.addPlayerTeam(name, passer)
            self.showAdd(name, passer, self.possession, playText, debug)
            

        keys = [" sacked by ", " sacked ", " Sacked By ", " SACKED BY "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            passer = playText[:pos].strip()
            self.addPlayerTeam(name, passer)
            self.showAdd(name, passer, self.possession, playText, debug)

    
    
    ################################################################################
    ## Runners
    ################################################################################
    def addRunner(self, playText, debug=False):
        name = "runner"
        keys = [" run for ", " Run For ", " RUN FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            runner = playText[:pos].strip()
            self.addPlayerTeam(name, runner)
            self.showAdd(name, runner, self.possession, playText, debug)
        else:
            run  = ("(run|Run|RUN)")
            num  = "([+-?]\d+|\d+)"  
            dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
            m = re.split("{0}\s{1}\s{2}".format(num, dist, run), playText)
            if len(m) > 1:
                runner = m[0].strip()
                self.addPlayerTeam(name, runner)
                self.showAdd(name, runner, self.possession, playText, debug)

    
    
    ################################################################################
    ## Receivers
    ################################################################################
    def addReceiver(self, playText, debug=False):
        name = "receiver"
        keys = ["pass complete", "Pass Complete", "PASS COMPLETE", "pass incomplete", "Pass Incomplete", "PASS INCOMPLETE"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            txt = playText[pos:].strip()
            txt = " ".join(txt.split()[2:])[3:].strip()
            keys = [" for ", " For ", " FOR "]
            pos = [x for x in [txt.find(x) for x in keys] if x != -1]
            if len(pos) > 0 and min(pos) > 0:
                receiver = txt[:min(pos)].strip()
                self.addPlayerTeam(name, receiver)
                self.showAdd(name, receiver, self.possession, playText, debug)

    
    
    ################################################################################
    ## Defense
    ################################################################################
    def addDefense(self, playText, debug=False):
        name = "defense"
        keys = [" sacked by ", " Sacked By ", " SACKED BY "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:            
            txt = playText[pos+11:].strip()
            keys = [" for ", " For ", " FOR "]
            pos = max([txt.find(x) for x in keys])
            if pos > 0:
                defense = txt[:pos].strip()
                self.addPlayerTeam(name, defense)
                self.showAdd(name, defense, self.possession, playText, debug)


    
    
    ################################################################################
    ## Kickers
    ################################################################################
    def addKicker(self, playText, debug=False):
        name = "kicker"
        keys = ["kickoff", "Kickoff", "KICKOFF"]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            kicker = playText[:pos].strip()
            self.addPlayerTeam(name, kicker)
            self.showAdd(name, kicker, self.possession, playText, debug)

    
    
    ################################################################################
    ## Field Goal Kicker
    ################################################################################
    def addFGKicker(self, playText, debug=False):   
        name = "fgkicker"         
        if playText[-1] == ")":
            pos = playText.rfind("(")
            if pos > 0:
                txt = playText[pos+1:-1]
                keys = [" kick", " Kick", " KICK"]
                pos = max([txt.find(x) for x in keys])
                if pos > 0:
                    fgkicker = txt[:pos].strip()
                    self.addPlayerTeam(name, fgkicker)
                    self.showAdd(name, fgkicker, self.possession, playText, debug)
            
            
        kick = ("(fg|FG)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
        m = re.split("{0}\s{1}\s{2}".format(num, dist, kick), playText)
        if len(m) > 1:
            fgkicker = m[0].strip()
            self.addPlayerTeam(name, fgkicker)
            self.showAdd(name, fgkicker, self.possession, playText, debug)
            
            
        wrd1 = ("(field|Field|FIELD)")
        wrd2 = ("(goal|Goal|GOAL)")
        num  = "([+-?]\d+|\d+)"  
        dist = ("(yards|yard|Yard|Yds|yds|Yd|yd)")
        
        m = re.split("{0}\s{1}\s{2}\s{3}".format(num, dist, wrd1, wrd2), playText)
        if len(m) > 1:
            fgkicker = m[0].strip()
            self.addPlayerTeam(name, fgkicker)
            self.showAdd(name, fgkicker, self.possession, playText, debug)

    
    
    ################################################################################
    ## Punters
    ################################################################################
    def addPunt(self, playText, debug=False):
        name = "punter"         
        keys = [" punt for ", " Punt For ", " PUNT FOR "]
        pos = max([playText.find(x) for x in keys])
        if pos > 0:
            punter = playText[:pos].strip()
            self.addPlayerTeam(name, punter)
            self.showAdd(name, punter, self.possession, playText, debug)


_, _ = clock("Last Run")

In [90]:
x = ['Joe', 'Red', 'Blue']
start = ["{0}.".format(y[0]) for y in x[:-1]]
end   = x[-1]
" ".join([" ".join(start), end])

'J. R. Blue'

In [ ]:
kick = ("(fg|FG)")
num  = "([+-?]\d+|\d+)"  
dist = ("(yards|yard|Yds|yds|Yd|yd)")
        
x = 'Matt Severino 47 yd FG MISSED'
m = re.split("{0}\s{1}\s{2}".format(num, dist, kick), x)
if len(m) > 1:
    fgkicker = m[0].strip()
    print(fgkicker)

In [ ]:
#x = keyplayers()
txt  = "Evan O'Hara kickoff for 58 yds"
kickoff = [" kickoff ", " Kickoff ", " KICKOFF "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Damien Harris run for 3 yds to the Alab 38"
kickoff = [" run ", " Run ", " RUN "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Tua Tagovailoa pass complete to Damien Harris for 14 yds to the Alab 46 for a 1ST down"
kickoff = [" pass ", " Pass ", " PASS "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())

txt = "Tua Tagovailoa sacked by G.G. Robinson for a loss of 6 yards to the Alab 32"
kickoff = [" sacked by ", " Sacked By ", " SACKED BY "]
pos = max([txt.find(x) for x in kickoff])
print(txt[:pos].strip())





In [ ]:

0 	 0 	 Evan O'Hara kickoff for 58 yds
0 	 1 	 Damien Harris run for 3 yds to the Alab 38
0 	 2 	 Tua Tagovailoa sacked by G.G. Robinson for a loss of 6 yards to the Alab 32
0 	 3 	 Tua Tagovailoa pass complete to Damien Harris for 14 yds to the Alab 46 for a 1ST down
0 	 4 	 Tua Tagovailoa pass complete to Irv Smith Jr. for 7 yds to the Lvile 47
0 	 5 	 Tua Tagovailoa pass complete to DeVonta Smith for 28 yds to the Lvile 19 for a 1ST down
0 	 6 	 Tua Tagovailoa run for 8 yds to the Lvile 11
0 	 7 	 Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds for a TD, (Austin Jones KICK)
1 	 0 	 Joseph Bulovas kickoff for 50 yds for a touchback
1 	 1 	 Jawon Pass pass complete to Micky Crum for 13 yds to the Lvile 38 for a 1ST down
1 	 2 	 Jawon Pass pass complete to Micky Crum for 5 yds to the Lvile 43
1 	 3 	 Trey Smith run for 2 yds to the Lvile 45
1 	 4 	 Jawon Pass pass incomplete to Trey Smith
1 	 5 	 Mason King punt for 35 yds, fair catch by Jaylen Waddle at the Alab 20
2 	 0 	 Josh Jacobs run for 6 yds to the Alab 26
2 	 1 	 Josh Jacobs run for 2 yds to the Alab 28
2 	 2 	 Tua Tagovailoa run for 7 yds to the Alab 35 for a 1ST down
2 	 3 	 Tua Tagovailoa pass incomplete
2 	 4 	 Tua Tagovailoa pass incomplete to Josh Jacobs
2 	 5 	 Tua Tagovailoa pass complete to DeVonta Smith for 26 yds to the Lvile 39 for a 1ST down
2 	 6 	 Najee Harris run for 4 yds to the Lvile 35
2 	 7 	 Tua Tagovailoa pass complete to DeVonta Smith for 18 yds DeVonta Smith fumbled, forced by Cornelius Sturghill, recovered by Lvile Dorian Etheridge
3 	 0 	 Jawon Pass pass incomplete to Seth Dawkins
3 	 1 	 Dae Williams run for 4 yds to the Lvile 21
3 	 2 	 LOUISVILLE Penalty, Substitution Infraction (-5 Yards) to the Lvile 16
3 	 3 	 Jawon Pass pass incomplete to Chatarius Atwell
3 	 4 	 Mason King punt for 39 yds, fair catch by Jaylen Waddle at the Alab 45
4 	 0 	 Damien Harris run for 32 yds to the Lvile 23 for a 1ST down
4 	 1 	 Tua Tagovailoa pass complete to Jaylen Waddle for 6 yds to the Lvile 17
4 	 2 	 Damien Harris run for 3 yds to the Lvile 14
4 	 3 	 Damien Harris run for 5 yds to the Lvile 9 for a 1ST down
4 	 4 	 Tua Tagovailoa pass incomplete to Jerry Jeudy
4 	 5 	 Tua Tagovailoa run for 9 yds for a TD, (Austin Jones KICK)
5 	 0 	 Joseph Bulovas kickoff for 65 yds for a touchback
5 	 1 	 ALABAMA Penalty, Defensive Pass Interference (Xavier McKinney) to the Lvile 40 for a 1ST down
5 	 2 	 Trey Smith run for 7 yds to the Lvile 47
5 	 3 	 Jawon Pass pass complete to Seth Dawkins for 3 yds to the 50 yard line for a 1ST down
5 	 4 	 Jawon Pass run for a loss of 2 yards to the Lvile 48
5 	 5 	 Jawon Pass pass complete to Trey Smith for 4 yds to the Alab 48
5 	 6 	 Jawon Pass run for 11 yds to the Alab 37 for a 1ST down
5 	 7 	 Jawon Pass pass complete to Dez Fitzpatrick for 13 yds to the Alab 24 for a 1ST down
5 	 8 	 LOUISVILLE Penalty, Substitution Infraction (-5 Yards) to the Alab 29
5 	 9 	 End of 1st Quarter
5 	 10 	 Dae Williams run for 3 yds to the Alab 26
5 	 11 	 Jawon Pass pass intercepted Deionte Thompson return for 25 yds to the Alab 25
6 	 0 	 Najee Harris run for 6 yds to the Alab 31
6 	 1 	 Tua Tagovailoa pass complete to Jerry Jeudy for 11 yds to the Alab 42 for a 1ST down
6 	 2 	 Tua Tagovailoa pass complete to Jaylen Waddle for 49 yds to the Lvile 9 for a 1ST down
6 	 3 	 Tua Tagovailoa run for 8 yds to the Lvile 1
6 	 4 	 Najee Harris run for no gain to the Lvile 1
6 	 5 	 LOUISVILLE Penalty, Substitution Infraction ( Yards) to the Lvile 1
6 	 6 	 Najee Harris run for 1 yd for a TD, (Austin Jones KICK)
7 	 0 	 Joseph Bulovas kickoff for 65 yds , Cornelius Sturghill return for 14 yds 

In [ ]:


            
    def fixDrive(self, gameID, driveNo, drive):
        if gameID == '401012747':
            if driveNo == 27:
                drive['HomeScore'] = ['10']
                
        if gameID == '401012795':
            if driveNo == 11:
                drive['Posession'] = ['12']
                
        if gameID == '401012777':
            if driveNo == 29:
                drive['Posession'] = ['204'] #'"ORST"]
                drive['HomeScore'] = ['41']

        return drive
    
            
    def insertMissingData(self, gameID, driveNo, fullDriveData, debug=False):
        if gameID == "401012295":
            if driveNo == 1:
                fullDriveData.plays[7].play.pa.runback = False                
        if gameID == "401012684":
            if driveNo == 27:
                fullDriveData.possession = "UNC"
                for i in range(len(fullDriveData.plays)):
                    fullDriveData.plays[i].possession = "UNC"
        if gameID == "401012350":
            if driveNo == 21:
                tmp = fullDriveData.plays[3]
                fullDriveData.plays[3] = fullDriveData.plays[4]
                fullDriveData.plays[4] = tmp
                
                ptype = playtype(fullDriveData.plays[3].play.text)
                play  = ptype.getPlay()
                play.analyze(debug=False)
                fullDriveData.plays[3].play = play
                ptype = playtype(fullDriveData.plays[4].play.text)
                play  = ptype.getPlay()
                play.analyze(debug=False)
                fullDriveData.plays[4].play = play
        if gameID == "401012348":
            if driveNo == 7:
                lastplay = fullDriveData.plays[-1]
                start    = lastplay.start
                playText = "Ryley Guay 37 yd FG GOOD"                
                ptype = playtype(playText)
                play  = ptype.getPlay()                
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                playStart = playstartclass(down=4, togo=start.togo, startY=start.startY, side=start.side)
                playStart.setQuarter(start.quarter)
                playStart.setGameClock(start.gameclock)
                playResults = playclass(possession=fullDriveData.possession, start=playStart, play=play)
                fullDriveData.plays.append(playResults)
            if driveNo == 8:
                lastplay = fullDriveData.plays[0]
                playText = "No Play"
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play                
        if gameID == "401012347":
            if driveNo == 0:
                lastplay = fullDriveData.plays[0]
                playText = lastplay.play.text
                playText = playText.replace("return for 17 yds", "return for 17 yds for a TD")
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
                lastplay.play.pa.touchdown = True
        if gameID == "401012341":
            if driveNo == 21:
                lastplay = fullDriveData.plays[-1]
                playText = lastplay.play.text
                playText = "{0} (Cooper Garcia Kick)".format(playText)                
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
        if gameID == "401012316":
            if driveNo == 14:
                lastplay = fullDriveData.plays[-1]
                lastplay.play.pa.fumble = False
                lastplay.play.pa.touchdown = True
        if gameID == "401012302":
            if driveNo == 15:
                lastplay = fullDriveData.plays[-2]
                lastplay.play.pa.fumble = False
                lastplay.play.pa.touchdown = True
        if gameID == "401012289":
            if driveNo == 20:
                lastplay = fullDriveData.plays[-2]
                playText = lastplay.play.text
                playText = playText.replace(" (", " For a TD (")             
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
                lastplay.play.pa.touchdown = True
        if gameID == "401012263":
            if driveNo == 18:
                lastplay = fullDriveData.plays[-2]
                playText = lastplay.play.text
                playText = "{0} For a TD".format(playText)            
                ptype = playtype(playText)
                play  = ptype.getPlay()
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                lastplay.play = play
        if gameID == "401012282":
            if driveNo == 8:
                lastplay = fullDriveData.plays[-1]
                start    = lastplay.start
                playText = "Tyrel Dodson 78 Yd Return of Blocked Field Goal (Seth Small Kick)"
                
                ptype = playtype(playText)
                play  = ptype.getPlay()                
                if play is None:
                    raise ValueError("Could not determine play type from {0}".format(playText))
                play.analyze(debug=False)
                play.pa.touchdown = True
                playStart = playstartclass(down=4, togo=start.togo, startY=start.startY, side=start.side)
                playStart.setQuarter(start.quarter)
                playStart.setGameClock(start.gameclock)
                playResults = playclass(possession=fullDriveData.possession, start=playStart, play=play)
                fullDriveData.plays.append(playResults)
                
                headline   = self.parseHeadline(["Touchdown"])
                fullDriveData.headline = headline


        return fullDriveData
            
            
    def augmentPlayWithScore(self, fullDriveData, fieldMap, debug=False):
        if not isinstance(fullDriveData, driveclass):
            raise ValueError("Game result is {0}".format(type(fullDriveData)))

        homeTeam = fieldMap["Home"]
        awayTeam = fieldMap["Away"]
        
        ## Ignore noplay plays
        plays      = [x for x in fullDriveData.plays if x.play.pa.getKey("noplay") is not True]
        headline   = fullDriveData.headline
        possession = fullDriveData.possession

        ## Check For Touchdown
        if isinstance(headline, touchdownplay):
            if sum(play.play.pa.touchdown for play in plays) == 0:
                for i in [(x+1)*-1 for x in list(range(0,len(plays),1))]:
                    if sum([x in plays[i].play.text for x in [" Kick)", " KICK)", " kick)", "2 point", " 2 Point", "Two-point", "two-point", "Two-Point"]]) > 0:
                        print("Artificially setting play [{0}] to touchdown since text does not reflect it: {1}".format(i, plays[i].play.text))
                        fullDriveData.plays[i].play.pa.touchdown = True
                        break
            
            if sum(play.play.pa.touchdown for play in plays) == 0:
                for i in [-1,-2]:
                    try:
                        lastplay = plays[i]
                    except:
                        continue

                    if lastplay.play.pa.getKey("end") is True:
                        continue

                    if lastplay.play.pa.getKey("penalty") is True or lastplay.play.pa.getKey("penaltyplay") is True:
                        continue

                    if lastplay.play.pa.getKey("fumble") is True:
                        lastplay.play.pa.touchdown = True
                        break
                        if possession == homeTeam:
                            lastplay.possession = awayTeam
                        elif possession == awayTeam:
                            lastpata.possession = homeTeam
                            
                    lastplay.play.pa.touchdown = True
                    print("Artificially setting play [{0}] to touchdown since text does not reflect it".format(i))
                    break
                    
                if sum(play.play.pa.touchdown for play in plays) == 0:
                    for j,play in enumerate(plays):
                        print(j,'\t',play.play.text)
                    raise ValueError("Touchdown still not in text!")


        ## Check For Fumble + Touchdown
        if isinstance(headline, fumbleplay):
            if headline.pa.getKey("touchdown"):
                if sum(play.play.pa.touchdown for play in plays) == 0:
                    print("Artificially setting last play to touchdown since text does not reflect it")
                    lastplay = plays[-1]
                    lastplay.play.pa.touchdown = True
                    if sum(play.play.pa.touchdown for play in plays) == 0:
                        raise ValueError("Touchdown still not in text!")
                
        return fullDriveData
        
        
            
            
    def analyzeGameScore(self, gameResult, fieldMap, debug=False):
        if not isinstance(gameResult, list):
            raise ValueError("Game result is {0}".format(type(gameResult)))

        homeTeam = fieldMap["Home"]
        awayTeam = fieldMap["Away"]

        finalHomeScore = None
        runningHomeScore = 0
        finalAwayScore = None
        runningAwayScore = 0
        if debug:
            print("{0: <7}{1: <5}{2: <4}{3: <4}{4: <4}{5: <4}".format("Drive", "POS", "H", "(H)", "A", "(A)"))
            
        for idr in range(len(gameResult)):
            driveData  = gameResult[idr]
            if isinstance(driveData, driveclass) is False:
                raise ValueError("Drive Data is not a DriveClass object!")
                
            homescore  = driveData.postdrivehomescore
            finalHomeScore = homescore
            awayscore  = driveData.postdriveawayscore
            finalAwayScore = awayscore
            
            drivePlays = driveData.plays
            if isinstance(drivePlays, list) is False:
                raise ValueError("Drive Plays is not a list object!")

            scoringplays = []
            for drivePlay in drivePlays:
                play       = drivePlay.play
                possession = drivePlay.possession
                score      = play.pa.isScore()
                if score > 0:
                    scoringplays.append(play.text)
                    if possession == homeTeam:
                        runningHomeScore += score
                    elif possession == awayTeam:
                        runningAwayScore += score
                    else:
                        raise ValueError("Possession {0} is neither home {1} or away {2}".format(possession, homeTeam, awayTeam))
                        
            if debug:
                print("{0: <7}{1: <5}{2: <4}{3: <4}{4: <4}{5: <4}{6}".format(idr, possession, runningHomeScore, homescore, runningAwayScore, awayscore, scoringplays))
            
        if finalAwayScore != runningAwayScore or finalHomeScore != runningHomeScore:
            return False
        return True
            
            
    def splitChangeOfPossession(self, gameResult, fieldMap, debug=False):
        if not isinstance(gameResult, list):
            raise ValueError("Game result is {0}".format(type(gameResult)))

        homeTeam = fieldMap["Home"]
        awayTeam = fieldMap["Away"]
        
        splitDrives = []
        for idr in range(len(gameResult)):
            driveData  = gameResult[idr]
            if isinstance(driveData, driveclass) is False:
                raise ValueError("Drive Data is not a DriveClass object!")
                
            
            drivePlays = driveData.plays
            if isinstance(drivePlays, list) is False:
                raise ValueError("Drive Plays is not a list object!")


            ######################################################
            ## Split play into [kick] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, kickoffplay) and playData.play.pa.getKey("noplay") is not True:
                    if driveData.possession == homeTeam:
                        playData.possession = awayTeam
                    elif driveData.possession == awayTeam:
                        playData.possession = homeTeam
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.kickoffplay = False
                    newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                    newReturnPlay.pa.fumble    = playData.play.pa.fumble
                    playData.play.pa.touchdown = False
                    playData.play.pa.fumble    = False
                    playData.play.pa.runback   = False
                    newPlayData = playclass(possession=driveData.possession, start=playData.start, play=newReturnPlay)
                    drivePlays.insert(dp+1, newPlayData)
                    break


            ######################################################
            ## Split play into [punt] + [return/safety + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, puntplay) and playData.play.pa.getKey("noplay") is not True:
                    if playData.play.pa.getKey("safety"):
                        newSafetyPlay  = safetyplay(text=playData.play.text)
                        newSafetyPlay.analyze()
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newSafetyPlay)
                        drivePlays.insert(dp+1, newPlayData)
                        break
                    else:
                        playData.play.pa.touchdown = False
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.pa.punt = True
                        newReturnPlay.analyze()
                        newReturnPlay.pa.punt = False
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if driveData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif driveData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break


            ######################################################
            ## Split play into [PAT] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if dp == len(drivePlays) - 1:
                    if isinstance(playData.play, patplay) and playData.play.pa.getKey("noplay") is not True:
                        if playData.play.pa.getKey("defpat"):
                            playData.play.pa.defpat  = False
                            playData.play.pa.runback = False
                            newReturnPlay  = returnplay(text=playData.play.text)
                            newReturnPlay.analyze()
                            newReturnPlay.pa.defpat = True
                            newReturnPlay.pa.pat    = True
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                            if driveData.possession == homeTeam:
                                newPlayData.possession = awayTeam
                            elif driveData.possession == awayTeam:
                                newPlayData.possession = homeTeam
                            drivePlays.insert(dp+1, newPlayData)
                            break


            ######################################################
            ## Split play into [interception] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, passingplay) and playData.play.pa.getKey("noplay") is not True:
                    if playData.play.pa.interception is True:
                        if dp == len(drivePlays) - 1:
                            newReturnPlay  = returnplay(text=playData.play.text)
                            newReturnPlay.analyze()
                            newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                            playData.play.pa.touchdown = False
                            newReturnPlay.pa.runback      = playData.play.pa.runback
                            playData.play.pa.runback   = False
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                            if playData.possession == homeTeam:
                                newPlayData.possession = awayTeam
                            elif playData.possession == awayTeam:
                                newPlayData.possession = homeTeam
                            drivePlays.insert(dp+1, newPlayData)
                            break


            ######################################################
            ## Split play into [fumble] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.play.pa.getKey("fumble") and playData.play.pa.getKey("runback"):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.fumble = False
                    newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                    playData.play.pa.touchdown = False
                    newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                    if playData.possession == homeTeam:
                        newPlayData.possession = awayTeam
                    elif playData.possession == awayTeam:
                        newPlayData.possession = homeTeam
                    drivePlays.insert(dp+1, newPlayData)
                    break


            ######################################################
            ## Split play into [FieldGoal] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, fieldgoalplay):
                    if playData.play.pa.getKey("blocked"):
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.pa.fieldgoal = True
                        newReturnPlay.analyze()
                        newReturnPlay.pa.fieldgoal = False
                        newReturnPlay.pa.touchdown = playData.play.pa.touchdown
                        playData.play.pa.touchdown = False
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if playData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif playData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break


            ######################################################
            ## Split play into [fumble] + [return + X] for end of game
            ######################################################
            if idr == len(gameResult) - 1:
                for dp in range(len(drivePlays)):
                    playData  = drivePlays[dp]
                    if playData.play.pa.getKey("fumble") is True and playData.play.pa.getKey("touchdown") is True:
                        playData.play.pa.touchdown = False
                        newReturnPlay  = returnplay(text=playData.play.text)
                        newReturnPlay.analyze()
                        newReturnPlay.pa.fumble = False
                        newReturnPlay.pa.touchdown = True
                        newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                        if playData.possession == homeTeam:
                            newPlayData.possession = awayTeam
                        elif playData.possession == awayTeam:
                            newPlayData.possession = homeTeam
                        drivePlays.insert(dp+1, newPlayData)
                        break



            ######################################################
            ## Split play into [safety] + [return + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if isinstance(playData.play, safetyplay):
                    newReturnPlay  = returnplay(text=playData.play.text)
                    newReturnPlay.analyze()
                    newReturnPlay.pa.safety    = False
                    newReturnPlay.pa.safetypts = True
                    newPlayData = playclass(possession=playData.possession, start=playData.start, play=newReturnPlay)
                    if driveData.possession == homeTeam:
                        newPlayData.possession = awayTeam
                    elif driveData.possession == awayTeam:
                        newPlayData.possession = homeTeam
                    drivePlays.insert(dp+1, newPlayData)
                    break
                    
                    
            ######################################################
            ## Split play into [TD] + [PAT + X]
            ######################################################
            for dp in range(len(drivePlays)):
                playData  = drivePlays[dp]
                if playData.play.pa.touchdown is True:
                    if sum([isinstance(x.play, patplay) for x in drivePlays]) == 0:
                        playData.play.pa.addPAT()
                        if playData.play.pa.getKey("addpat") is True:
                            playData.play.pa.addpat = False
                            newPATPlay  = patplay(text=playData.play.text)
                            newPATPlay.analyze()
                            newPATPlay.pa.touchdown = False
                            newPlayData = playclass(possession=playData.possession, start=playData.start, play=newPATPlay)
                            drivePlays.insert(dp+1, newPlayData)
                            break
                            
                                
        return gameResult
            
    
    
    def analyzeGameData(self, gameResult, debug=False):
        if not isinstance(gameResult, list):
            raise ValueError("Game result is {0}".format(type(gameResult)))
            
        print("Drive\tPlay\tPosess\t{0: <5}{1: <5}{2: <10}{3: <3}{4: <4}{5: <5}{6: <5}{7: <30}{8}".format("Gain", "Dist", "Class", "D", "TG", "Side", "Line", "Keys", "Text"))
        for idr,driveData in enumerate(gameResult):
            drivePlays = driveData.plays
            possession = driveData.possession
            headline   = driveData.headline
            detail     = driveData.detail
            print("\n\n")
            print(headline,'\t\t',detail)
            for ip,playData in enumerate(drivePlays):
                playPossession = playData.possession
                print("  {0}\t{1}\t{2}".format(idr,ip,playPossession), end="\t")
            
                keys = ", ".join([k for k,v in playData.play.pa.__dict__.items() if v is True])
                
                    
                try:
                    playResult   = playData.play.result
                    print("{0: <5}".format(playResult), end="")
                except:
                    print("{0: <5}".format(""), end="")
                
                    
                try:
                    playDist   = playData.start.distToEndZone
                    print("{0: <5}".format(playDist), end="")
                except:
                    print("{0: <5}".format(""), end="")
                    
                    
                try:
                    playName   = playData.play.name
                    print("{0: <10}".format(playName), end="")
                except:
                    print("{0: <10}".format(""), end="")
                    
                    
                try:
                    playDown   = playData.start.down
                    print("{0: <3}".format(playDown), end="")
                except:
                    print("{0: <3}".format(""), end="")
                    
                    
                try:
                    playTogo   = playData.start.togo
                    print("{0: <4}".format(playTogo), end="")
                except:
                    print("{0: <4}".format(""), end="")
                    
                    
                try:
                    playSide   = playData.start.side
                    print("{0: <5}".format(playSide), end="")
                except:
                    print("{0: <5}".format(""), end="")
                    
                    
                try:
                    playStartY   = playData.start.startY
                    print("{0: <5}".format(playStartY), end="")
                except:
                    print("{0: <5}".format(""), end="")

                

                print("{0: <30}".format(keys), end="")


                playText   = playData.play.text
                print("{0}".format(playText))



In [495]:
playText = " x (Austin Jones KICK)"
print(playText[-1])
        pos = 
        print(txt:)


IndentationError: unexpected indent (<ipython-input-495-7dd4bdee93be>, line 3)

In [823]:
def siground(x, sig=3):
    if not isinstance(x, (float, int)):
        print("Must pass a real number. You passed a {0}".format(type(x)))
        return None
    
    retval = '%s' % float('%.{0}g'.format(sig) % x)
    
    try:
        result = float(retval)
    except:
        print("Could not turn [{0}] into a float.".format(retval))
        result = None
    
    return result